In [171]:
import pandas as pd
import numpy as np
import re
from openpyxl import load_workbook

In [172]:
# Load all the necessary Excel files to notebook
address_df = pd.read_excel('recon_addresses.xlsx',sheet_name='Sheet1') # addresses for contractor groups

#dictionary for address parsing (street names and arteries)
street_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Street Names') 
artery_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Art')

#file for contractor group
first_df = pd.read_excel('Copy of reconn year to date.xlsx',sheet_name ='con_ed_sli_-_field_inspection_r')

#file with default values and data standardization 
mapping_df = pd.read_excel('mapping2.xlsx',sheet_name = 'RECONN')

#blank template to convert to 
new_df = pd.read_excel('SLI_Template.xlsx')

In [173]:
df = address_df.rename(columns={'HouseNumber': 'raw address'})
addresses = df['raw address'].tolist()

def process_data(address):
    if not isinstance(address, str):
      return ['', '', '', '', '']

    house_number_re = re.compile(r'^[\d-]+')
    compass_point_re = re.compile(r'(?<= )([NESW]|North|East|South|West)(?= )', re.IGNORECASE)
    street_name_re = re.compile(r'(?:(?<=[NESW] )|(?<= ))[\w\s-]+(?= (St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard))', re.IGNORECASE)
    artery_re = re.compile(r'(?<= )(St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard)', re.IGNORECASE)
    zip_code_re = re.compile(r'\b\d{5}\b')

    compass_dict = {
    's': 'South',
    'n': 'North',
    'e': 'East',
    'w': 'West',
    }

    artery_dict = {
    'st': 'Street',
    'av': 'Avenue',
    'ave': 'Avenue',
    'rd': 'Road',
    'blvd':'Boulevard'
    }

    address = re.sub(r'United States', '', address, flags=re.IGNORECASE)
    address = re.sub(r'New York', '', address, 1, flags=re.IGNORECASE) if address.lower().count('new york') > 1 else address

    house_number = house_number_re.search(address)
    house_number = house_number.group() if house_number else ''

    compass_point = compass_point_re.search(address)
    if compass_point:

      compass_point = compass_point.group()
      oc = compass_point
      compass_point = compass_dict.get(compass_point.lower(),compass_point)
    else:
      compass_point= ''

    street_name = street_name_re.search(address)
    if street_name:
        street_name = street_name.group().strip()
        # If the street name starts with a compass point, remove it
        if compass_point and street_name.startswith(oc):
            street_name = street_name[len(oc)+1:].strip() # +1 added to handle the space after compass point.
    else:
        street_name = ''

    artery = artery_re.search(address)
    if artery:
        artery = artery.group()
        artery = artery_dict.get(artery.lower(), artery)
    else:
        artery = ''

    zip_code = zip_code_re.search(address)
    zip_code = zip_code.group() if zip_code else ''

    return [house_number, compass_point, street_name, artery, zip_code]


formatted_addresses = [process_data(address) for address in addresses]
# print(formatted_addresses)

df_new = pd.DataFrame(formatted_addresses, columns=['HouseNumber', 'CompassPoint', 'StreetName', 'Artery', 'ZipCode'])

# concatenate the original DataFrame with the new one
# df_final = pd.concat([df, df_new], axis=1)

# df_new.to_excel('RECONN_ADRESS_PARSED.xlsx', index=False)

# Load the workbook
book = load_workbook('reconn_parsed.xlsx')

# Create a pandas Excel writer object
with pd.ExcelWriter('reconn_parsed.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:            
            df_new.to_excel(writer, 'Sheet1', index=False, header=True,startcol=10)

# Write df_final to the Excel file starting at column K
df_new.to_excel("RECONN_ADDRESS_PARSED", index=False)


ValueError: No engine for filetype: ''

In [ ]:

first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)

#
reconn = "meter_location_atmosphere_select_all_that_apply"
first_df[reconn] = first_df[reconn].str.split(';').str[0]
first_df[reconn] = first_df[reconn].str.split(',').str[0]

# create a dictionary from the mapping_df file
mapping_dict = mapping_df.set_index('normalized name')['column name'].to_dict()

default_values_dict = mapping_df.set_index('normalized name')['default value'].apply(lambda x: x.split(',') if isinstance(x, str) else x).to_dict()

new_df = pd.DataFrame(columns = new_df.columns)

In [ ]:
for column in new_df.columns:
    if column in mapping_dict and mapping_dict[column] in first_df.columns:
        column_name = mapping_dict[column]
        new_df[column] = first_df[column_name]
    else:
        new_df[column]=new_df[column]

In [ ]:
new_df['BuildingAccessible'] = new_df['BuildingAccessible'].fillna('YES')
new_df['OutsideSet'] = new_df['OutsideSet'].fillna('NO')
new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].fillna('YES')
new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].fillna('SINGLE')
new_df['MeterLocation'] = new_df['MeterLocation'].fillna('BASEMENT')
new_df['MeterLocationAtmosphere'] = new_df['MeterLocationAtmosphere'].fillna('METERROOM')
new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].fillna('METALLICBARE')
new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].fillna('METALLICBARE')
new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].fillna('METALLICBARE')
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].fillna('YES')
new_df['NoAccessReasonCdBuilding'] = new_df['NoAccessReasonCdBuilding'].fillna('OTHER')
new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].fillna('YES')
new_df['AccessibleCustomer'] = new_df['AccessibleCustomer'].fillna('YES')
new_df['NoAccessReasonCdCustomer'] = new_df['NoAccessReasonCdCustomer'].fillna('DIDNOTATTEMPT')
new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].fillna('NONE')
new_df['CorrosionLocationsCdCustomer'] = new_df['CorrosionLocationsCdCustomer'].fillna('NONE')
new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].fillna('OES')
new_df['HeadOfServiceValveNotAccessibleHeight'] = new_df['HeadOfServiceValveNotAccessibleHeight'].fillna('NO')
new_df['GasTurnedOff'] = new_df['GasTurnedOff'].fillna('NO')
new_df['AllServicePipingInspected'] = new_df['AllServicePipingInspected'].fillna('YES')
new_df['CorrosionFoundOnCdCustomer'] = new_df['CorrosionFoundOnCdCustomer'].fillna('OTHER')
new_df['ImproperConditionsFoundCustomer'] = new_df['ImproperConditionsFoundCustomer'].fillna('NO')
new_df['DamagedGasMeterCustomerCd'] = new_df['DamagedGasMeterCustomerCd'].fillna('OTHER')
new_df['InappropriateFlexibleConnectorCds'] = new_df['InappropriateFlexibleConnectorCds'].fillna('THROUGH_WALL')
new_df['AccessibleConEd'] = new_df['AccessibleConEd'].fillna('YES')
new_df['NoAccessReasonCdConed'] = new_df['NoAccessReasonCdConed'].fillna('DIDNOTATTEMPT')
new_df['POESleeveLocationCd'] = new_df['POESleeveLocationCd'].fillna('OTHER')
new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].fillna('YES')
new_df['ImproperConditionsFoundConEd'] = new_df['ImproperConditionsFoundConEd'].fillna('NO')
new_df['ImproperSubstandardConditionConEdCds'] = new_df['ImproperSubstandardConditionConEdCds'].fillna('OTHER')
new_df['DamagedGasMeterConEdCd'] = new_df['DamagedGasMeterConEdCd'].fillna('OTHER')
new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].fillna('NO')
new_df['SupervisorNotified'] = new_df['SupervisorNotified'].fillna('YES')
new_df['PlasticExposed'] = new_df['PlasticExposed'].fillna('NO')
new_df['PlasticExtendsIntoStructure24Inches'] = new_df['PlasticExtendsIntoStructure24Inches'].fillna('YES')
new_df['CorrosionSeverityCdConEd'] = new_df['CorrosionSeverityCdConEd'].fillna('NONE')
new_df['CorrosionFoundOnCdConEd'] = new_df['CorrosionFoundOnCdConEd'].fillna('OTHER')
new_df['CorrosionLocationsCdConEd'] = new_df['CorrosionLocationsCdConEd'].fillna('OTHER')
new_df['NGDRemovedReasonCd'] = new_df['NGDRemovedReasonCd'].fillna('OTHER')
new_df['PoeAccessible'] = new_df['PoeAccessible'].fillna('YES')
new_df['PoeNoAccessReasonCd'] = new_df['PoeNoAccessReasonCd'].fillna('OTHER')
new_df['PoeFoundSealedCondition'] = new_df['PoeFoundSealedCondition'].fillna('NO')
new_df['PoeLeftSealedCondition'] = new_df['PoeLeftSealedCondition'].fillna('NO')
new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].fillna('OTHER')
new_df['GasOdorPresent'] = new_df['GasOdorPresent'].fillna('NO')
new_df['GasLeakCorrosionRelated'] = new_df['GasLeakCorrosionRelated'].fillna('NO')
new_df['LeakItemCd'] = new_df['LeakItemCd'].fillna('OTHER')

In [ ]:
def apply_mapping(df, column_name, value_mapping):
    """Applies value mapping to a column, handles missing values, and retrieves defaults."""
    mapping = value_mapping.get(column_name, {})
    default_value = mapping.get("default_value", "OTHER")
    df[column_name] = df[column_name].apply(
    lambda x: mapping.get(x, default_value) if x and pd.notna(x) else 'default_value'
    )

In [ ]:
value_mapping = {
    'OutsideSet':{
        'METERSET INSIDE':'NO',
        'yes':'YES',
        'no':'NO',
        #'0':''
    },

    'MeterLocationAtmosphere':{
        # 'METER AT/NEAR POE;':'METERPOE',
        # 'BELOW GRADE;':'BELOWGRADE',
        # 'AIR CONDITIONING SPACE;':'ACSPACE',
        # 'CLOSED TO THE OUTSIDE;':'CLOSED',
        # 'HEATED SPACE;':'HEATSPACE',
        # 'ROOM SET;':'ROOMSET',
        # 'VENTED TO THE OUTSIDE;':'VENTED',
        # 'METER ROOM;':'METERROOM',
        'BELOW GRADE':'BELOWGRADE',
        'HEATED SPACE':'HEATSPACE',
        'CLOSED TO THE OUTSIDE':'CLOSED',
        'METER AT/NEAR POE':'METERPOE',
        'ROOM SET':'ROOMSET',
        'VENTED TO THE OUTSIDE':'VENTED',
        'METER ROOM':'METERROOM',
        'AIR CONDITIONING SPACE;':'ACSPACE',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'BELOWGRADE',
        'Air Conditioning Space,Closed to Outside,Heated Space':'VENTED',
        'Below grade (Basement, Crawl Space, ect)':'BELOWGRADE',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Apartments ,Closed to Outside,Heated Space':'OTHER',
        'Closed to Outside':'CLOSED',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside':'OTHER',
        'Air Conditioning Space,Heated Space':'OTHER',
        '0':'',
        'Heated Space,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Heated Space,Vented to Outside':'OTHER',
        'Air Conditioning Space,Closed to Outside,Heated Space,Vented to Outside':'OTHER',
        'Closed to Outside,Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),apartment ':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),Vented to Outside':'OTHER',
        'Kitchen ':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Vented to Outside':'OTHER',
        'Apartment ':'ROOMSET',
        'Air Conditioning Space':'ACSPACE',
        'Apartment ,Below grade (Basement, Crawl Space, ect)':'OTHER',
        'Closed to Outside,Heated Space,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Other':'BELOWGRADE',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Heated Space,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Garage ':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect)':'OTHER',
        'Vented to Outside,garage ':'VENTED',
        'Air Conditioning Space,Closed to Outside,Vented to Outside':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Kitchens':'OTHER',
        'ground floor ':'BELOWGRADE',
        'Heated Space':'HEATSPACE',
        'In apartments and outside ':'OTHER',
        'Closed to Outside,Vented to Outside':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,One outside met set':'OTHER',
        ' No meters in the building.':'',
        'Apartment ,Closed to Outside':'OTHER',
        'Air Conditioning Space,Closed to Outside':'OTHER',
        'Apartments ,Closed to Outside':'OTHER',
        'Closed to Outside,Heated Space,Other':'OTHER',
        'Vented to Outside':'VENTED',
        'Air Conditioning Space,Heated Space,Vented to Outside':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Vented to Outside':'OTHER',
        'No meters, no gas service. Poe is capped, locked and off':'',
        'Meter room':'METERROOM',
        'Garage':'ROOMSET',
        'Utility room,Vented to Outside':'OTHER',
        'Apt ,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Apt 2B  Kitchen ':'OTHER',
        'unknown':'',
        'Apt':'ROOMSET',
        'Vented to Outside,garage':'OTHER',
        'outside meter':'OTHER',
        'APT4A':'METERROOM',
        'Air Conditioning Space,Apt ,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Apartment 2,Closed to Outside':'ROOMSET',
        'Apartment 53,Closed to Outside,Heated Space':'ROOMSET',
        'Apartments ':'ROOMSET',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Outside ':'OTHER',
        'Basement ':'BELOWGRADE',
        'Apartments,Below grade (Basement, Crawl Space, ect)':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,RESTAURANT BASEMENT “PIO PIO”':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),In apartments ':'OTHER',
        'In apartments ':'ROOMSET',
        'Closed to Outside,Heated Space,In the apartments ':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,IN RESTAURANTS BASEMENT “PIO PIO”':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Outside ':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,One meter outside, boiler ':'OTHER',
        'Apartment complex hallways ,Closed to Outside,Heated Space':'OTHER',
        'Closed off in cabinet':'CLOSED',
        'Apartments,Closed to Outside':'OTHER',
        'Other':'OTHER',
        'No service at this location ':'',
        'Below grade (Basement, Crawl Space, ect),Heated Space,PARKING GARAGE. NEAR POE':'OTHER',
        'Heated Space,parking garage':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),Heated Space,Other':'OTHER',
        'Inside commercial space on second floor and on':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Two separate Poe ':'OTHER',
        'NO METERS, BUILDING IS VACANT.':'',
        'Apartments ,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Closed to Outside,Other':'CLOSED',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Vented to Outside':'OTHER',
        'Park storage room ':'ROOMSET',
        'First floor':'ROOMSET',
        'Boiler room upstairs ':'ROOMSET',
        '2nd floor':'ROOMSET',
        'Closed to Outside,Under stairs':'CLOSED',
        'Closed to Outside,Multiple locations.':'CLOSED',
        'outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Service entrance ':'BELOWGRADE',
        'No meter found ':'',
        'Doorway of church ,Vented to Outside':'OTHER',
        'Ground floor office ':'ROOMSET',
        'Closed to Outside,closet in room':'OTHER',
        'No meter found':'',
        'Garage ':'ROOMSET',
        'THERE IS NO METERS IN THIS LOCATION.( TUFINO A - 10/12/2023 )':'',
        'Ground floor':'ROOMSET',
        'Store':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Meter are disconnected and on floor ':'BELOWGRADE',
        'Building under construction. Poe is off and locked. No gas service to building ':'',
        'Ground floor meter room ':'METERROOM',
        'Ground floor meter room ,Heated Space':'METERROOM',
        'Closed to Outside,Heated Space,Meter room ':'CLOSED',
        'Closed to Outside,Hallway ,Heated Space':'CLOSED',
        'Closed to Outside,Outside trap door ':'CLOSED',
        'Below grade (Basement, Crawl Space, ect),Under stairs ,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),hallways':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Garage ,Vented to Outside':'OTHER',
        'Ground floor ,Heated Space':'OTHER',
        'Air Conditioning Space,Ground floor ':'ACSPACE',
        'Air Conditioning Space,Ground floor supermarket ':'ACSPACE',
        'There is no NBD meters in this location ':'',
        'Air Conditioning Space,Garage ':'ACSPACE',
        '1st floor utility room,Closed to Outside,Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,vault under stairs':'OTHER',
        'Below grade (Basement, Crawl Space, ect),vault':'OTHER',
        'Air Conditioning Space,Closed to Outside,kitchen ':'OTHER',
        'Closed to Outside,Heated Space,Street level ':'OTHER',
        'Closed to Outside,Under steps':'OTHER',
        'No meters ':'',
        'Gas meter room ground level':'METERROOM',
        'Air Conditioning Space,Vented to Outside':'OTHER',
        'Closed to Outside,Kitchen ':'CLOSED',
        'Air Conditioning Space,Closed to Outside,hallway':'ACSPACE',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Heated Space,apartment ':'OTHER',
        'Inside restaurant':'OTHER',
        'Kitchen ,Vented to Outside':'OTHER',
        'In apartment ':'ROOMSET',
        'No gas meter at location':'',
        'In apartment':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Other':'BELOWGRADE',
        'Poe is locked,off and plugged ':'',
        'Outside ':'OTHER',
        'Street level':'OTHER',
        'No meter':'',
        'Below grade (Basement, Crawl Space, ect),Outside Meter Set':'BELOWGRADE',
        'Heated Space,gas closet in laundry room ':'HEATSPACE',
        'Closed to Outside,garage':'CLOSED',
        'Garage meter room ,Heated Space':'METERROOM',
        'Below grade (Basement, Crawl Space, ect),Under stairs outside,Vented to Outside':'BELOWGRADE',
        'There’s is no longer a meter in this building ':'',
        'No meters in basement, meters in apartments no access ':'',
        'Meters in apartments no access ':'',
        'Apartments':'ROOMSET',
        'Under stairs ,Vented to Outside':'VENTED',
        'store ':'OTHER',
        'Meter not present ':'',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Garage ,Heated Space':'OTHER',
        'No meters':'',
        'Apartments,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Closed to Outside,Meters are in apartments ':'OTHER',
        'Fine fare supermarket ground floor ':'OTHER',
        '2 poes in building ,Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'No meters in building. Building under construction ':'',
        'Building under construction no meters in building ':'',
        'Closed to Outside,Garage ':'CLOSED',
        'Closed to Outside,Hallway,Vented to Outside':'CLOSED',
        'Closed to Outside,Meter Room':'CLOSED',
        'Closed to Outside,Hallway':'CLOSED',
        'Hallway Storage,Vented to Outside':'VENTED',
        'Closed to Outside,Meter Room/ Under Stoop,Vented to Outside':'CLOSED',
        'Bathroom,Below grade (Basement, Crawl Space, ect),Closed to Outside,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Meter Room':'OTHER',
        'Air Conditioning Space,Closed to Outside,Garage/Meter room,Vented to Outside':'OTHER',
        'Air Conditioning Space,Closed to Outside,Meter Room,Vented to Outside':'OTHER',
        'Closed to Outside,Meter Room ,Vented to Outside':'CLOSED',
        'Closed to Outside,Meter Room ':'CLOSED',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Laundry room ':'OTHER',
        'Closed to Outside,Hallway Lower Level':'CLOSED',
        'Ground floor ,Vented to Outside':'VENTED',
        'Closed to Outside,Ground floor ':'CLOSED',
        'Air Conditioning Space,Heated Space,Other,Vented to Outside':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Other':'OTHER',
        'Hallways. Located next to apartment doors ,Vented to Outside':'OTHER',
        'Ground floor,Vented to Outside':'VENTED',
        'Outside shed ,Vented to Outside':'VENTED',
        'Apartment ,Heated Space':'HEATSPACE',
        'Above ground apartment ,Closed to Outside':'CLOSED',
        'Closed to Outside,In apartment ':'CLOSED',
        '1st fl meter room ,Closed to Outside'
        'Above floor,Vented to Outside':'VENTED',
        '1st floor closet ,Closed to Outside':'CLOSED',
        'room sets':'ROOMSET',
        'No access ':'',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,One is outside ,Vented to Outside':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,One meter is outside it’s for the boiler,Vented to Outside':'OTHER',
        'In apartments 8/1/23':'ROOMSET',
        'Air Conditioning Space,At ground level in restaurant ':'ACSPACE',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,Meters in apartments ':'OTHER',
        'Air Conditioning Space,On ground level in restaurant Poe is a riser':'OTHER',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,In Apartments ':'OTHER',
        'first floor ':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Sub basement ':'BELOWGRADE',
        'ground floor. door right. ':'OTHER',
        'No meters in building ':'',
        'warehouse mechanic ':'OTHER',
        'Heated Space,closet in kitchen ':'HEATSPACE',
        'Closed to Outside,Hallway,Heated Space':'OTHER',
        '2 meters on 2nd and 3rd floor ,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space,On 41st Street side and on 42nd Street side ':'OTHER',
        'No meter in building ':'',
        '2nd fl,Air Conditioning Space':'ACSPACE',
        'meter removed ':'',
        'Apartment':'ROOMSET',
        'No gas or meter in building,Poe plugged and locked ':'',
        'ground floor. ':'',
        'No meter in building,Poe is locked ':'',
        'Hallway':'',
        'Outside set':'OTHER',
        'No gas in building ':'',
        'Ap,Heated Space':'HEATSPACE',
        'Closed to Outside,Heated Space,Vented to Outside,garage ':'OTHER',
        'Other in apartments ':'ROOMSET',
        'Garage ,Heated Space':'HEATSPACE',
        'Closed to Outside,Garage ,Heated Space':'OTHER',
        'Apt ,Below grade (Basement, Crawl Space, ect),Heated Space':'OTHER',
        'Closed to Outside,apartment ':'CLOSED',
        '1st floor ':'OTHER',
        'Heated Space,Room set':'OTHER',
        'In the office of the church ':'OTHER',
        'In the apartments ':'ROOMSET',
        'Meter room ':'METERROOM',
        'Apt ':'ROOMSET',
        'Stair case':'OTHER',
        'Bathroom inside every apartment ':'ROOMSET',
        'Kitchen':'ROOMSET',
        'Closed to Outside,Heated Space,garage ':'OTHER',
        'Fire pump room in backyard ,Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Heated Space,Poe in basement of the bakery ':'OTHER',
        'No meter locked at hos customer waiting on meter to be installed ':'OTHER',
        'auto shop ':'OTHER',
        'Bathroom':'ROOMSET',
        'appointment ':'OTHER',
        'Apt ,Closed to Outside':'CLOSED',
        'Closed to Outside,Ground floor ,Heated Space':'OTHER',
        'Closet in back of first floor':'OTHER',
        'Behind counter inside car wash':'OTHER',
        '1st floor hallway ':'OTHER',
        'Hallway closet ':'OTHER',
        'Apartment 8,Below grade (Basement, Crawl Space, ect)':'BELOWGRADE',
        'Closet':'OTHER',
        'Hallways ':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Hallway of building 622 & in basement ':'BELOWGRADE',
        'Hallway ':'OTHER',
        'Vaulted area outside set ,Vented to Outside':'OTHER',
        'No meters on site. Off and capped and service valve.':'',
        'Closed to Outside,Closet in hallway':'CLOSED',
        'Apt 10d':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),CUSTOMER APARTMENT ':'OTHER',
        'Air Conditioning Space,Apartments,Below grade (Basement, Crawl Space, ect),Heated Space,Vented to Outside':'OTHER',
        'Air Conditioning Space,Apartments,Below grade (Basement, Crawl Space, ect),Closed to Outside,Heated Space':'OTHER',
        'Air Conditioning Space,Closed to Outside,Hallways,Heated Space':'OTHER',
        'Air Conditioning Space,Apartments ':'ACSPACE',
        'Apartments ,Below grade (Basement, Crawl Space, ect),Closed to Outside':'OTHER',
        'Closed to Outside,Ground floor,Heated Space':'OTHER',
        'Cleaners ground floor ,Heated Space':'OTHER',
        'Hallway,Heated Space':'HEATSPACE',
        'warehouse garage ':'OTHER',
        'Apts ,Below grade (Basement, Crawl Space, ect),Heated Space':'OTHER',
        'Bathroom ,Below grade (Basement, Crawl Space, ect),Heated Space':'OTHER',
        'warehouse':'OTHER',
        'Closed to Outside,Garage,Heated Space':'OTHER',
        'car shop/ room ':'OTHER',
        'Vented to Outside,warehouse ':'VENTED',
        'warehouse ':'OTHER',
        'Air Conditioning Space,Hallway ':'ACSPACE',
        'Air Conditioning Space,Closed to Outside,Heated Space,Office':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Heated Space,Storage room':'OTHER',
        'Air Conditioning Space,Heated Space,Restaurant ':'OTHER',
        'Apartment ,Closed to Outside,Heated Space':'OTHER',
        'Apartments,Heated Space':'HEATSPACE',
        'Heated Space,Kitchen ':'HEATSPACE',
        'One meter located in apt apt 2 POE IN RESTAURANT NEXT DOOR  REST METER LOCATED IN BASEMENT ':'OTHER',
        'Air Conditioning Space,Apartment':'ACSPACE',
        'Ground floor,Heated Space':'HEATSPACE',
        'Apartment,Closed to Outside':'CLOSED',
        'Heated Space,Other':'HEATSPACE',
        'Heated Space,Meter room ':'HEATSPACE',
        'Closed to Outside,Heated Space,Meters located in the apartments.':'OTHER',
        'Above Ground':'OTHER',
        'Air Conditioning Space,Closed to Outside,Under stoop':'OTHER',
        'Closed to Outside,Under stoop ':'CLOSED',
        'Closed to Outside,Side room ':'CLOSED',
        'Apt 2a':'ROOMSET',
        'Apt 9a':'ROOMSET',
        'Apt ,Closed to Outside,Heated Space':'OTHER',
        'Below grade (Basement, Crawl Space, ect),Some meters in basement others in apartments':'OTHER',
        'Indoor 1a':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),Superintendent shop':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),Heated Space,Steam equipment room ':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),Bsmt oil tank room':'BELOWGRADE',
        'In apartment 2G kitchen ':'ROOMSET',
        'Apartment 10a':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),Trap door on 1343':'BELOWGRADE',
        'Basement heated space ':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),One meter located in the basement and the other meters are located in the apartments.':'BELOWGRADE',
        'Heated Space,Kitchen of apartment ':'HEATSPACE',
        'Bathroom ':'ROOMSET',
        'Air Conditioning Space,Heated Space,Other':'OTHER',
        'Garage,Vented to Outside':'VENTED',
        'Air Conditioning Space,Below grade (Basement, Crawl Space, ect),Heated Space,Other':'OTHER',
        'Under stoop':'OTHER',
        'Laundry room':'ROOMSET',
        'on ground floor ':'OTHER',
        'Rec room of park':'ROOMSET',
        'Air Conditioning Space,Closed to Outside,Heated Space,Other':'OTHER',
        'Laundry room ':'ROOMSET',
        'in hallway ':'OTHER',
        'back room. ground floor ':'ROOMSET',
        'Meter room ,Vented to Outside':'METERROOM',
        'Below grade (Basement, Crawl Space, ect),Boiler room':'BELOWGRADE',
        'Closed to Outside,Heated Space,On the 1st floor ':'OTHER',
        'Hallway ,Vented to Outside':'VENTED',
        '5th Floor Rooom':'ROOMSET',
        'Below grade (Basement, Crawl Space, ect),In apartment ':'BELOWGRADE',
        'Rear end basement':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),Garage Meter Room':'BELOWGRADE',
        'Below grade (Basement, Crawl Space, ect),Vented to Outside,garage ':'BELOWGRADE',
        'Garage ,Vented to Outside':'VENTED',
        'Meters are located outside ':'OTHER',
        'outside meter set':'OTHER',
        'Air Conditioning Space,Closed to Outside,Heated Space,apt':'OTHER',
        'Apartment ,Below grade (Basement, Crawl Space, ect),Closed to Outside':'OTHER',
        'Basement closet ,Below grade (Basement, Crawl Space, ect),Closed to Outside':'OTHER',
        'Closed to Outside,Ground Floor':'CLOSED',
        'Heated Space,apt':'HEATSPACE',
        'apartment cabinet':'OTHER',
        'Closet ':'OTHER',
        'Closed to Outside,Garage ,Vented to Outside':'OTHER',
        'Hallway in a closet ':'OTHER',
    },
    
    'NoAccessReasonCdBuilding':{
        'ACCESS DENIED COVID':'CORONAVIRUS',
        'ADRESS DOES NOT EXIST':'ADDRESSNOTEXIST',
        'BUILDING QUARANTINED':'QUARANTINED',
        'CATS REMARK':'CATS',
        'CUST FAILED KEEP APT':'CUSTFAILAPPT',
        'CUST NOT AT HOME':'CUSTNOTHOME',
    },
    
    'CorrosionLocationsCdCustomer':{
        'BASEMENT;':'BASEMENT',
        'NONE;':'NONE',
        'APARTMENT ;':'APARTMENT',
        'GARAGE;':'GARAGE',
        'HALLWAY;':'HALLWAY',
        'UNDER STAIRS;':'UNDER_STAIRS',
        'VAULT;':'VAULT',
        'NONE;':'NONE',
    },
            
    'CtagEquipmentCd':{
        'Other':'O',
        'EXTENSION OF SERVICE':'OES',
    },
    
    'DamagedGasMeterCustomerCd':{
        'BENT GAS PIPING':'BENT_GAS_PIPING',
    },
    'NoAccessReasonCdConed':{
        'Access Denied - COVID':'CORONAVIRUS',
        'Address Does Not Exist':'ADDRESSNOTEXIST',
        'CATS Remark':'CATS',
        'Customer failed to keep appt ':'CUSTFAILAPPT',
        'Customer Not Home':'CUSTNOTHOME',
        'did not Attempt':'DIDNOTATTEMPT',
    },
        
    'DamagedGasMeterConEdCd':{
        'BENT_GAS_PIPING':'BENT_GAS_PIPING',
    },
    
    'CorrosionFoundOnCdConEd':{
        'EXTENSION':'OTHER',
        'HOUSE PIPE':'OTHER',
        'SERVICE HEAD ADAPTER':'SERVICE_HEAD_ADAPTER',
    },
    'CorrosionLocationsCdConEd':{
        'APARTMENT ;':'APARTMENT',
        'BASEMENT;':'BASEMENT',
        'GARAGE;':'GARAGE',
        'HALLWAY;':'HALLWAY',
        'NONE;':'NONE',
        'UNDER STAIRS;':'UNDER_STAIRS',
        'VAULT;':'VAULT',
    },
    
    'LeakItemCd':{
        'METER;':'METER',
        'VALVE;':'VALVE',
        'REGULATOR;METER;':'REGULATOR',
        'FITTING THREADS;':'FITTINGTHREAD',
        'FITTING (ELBOW, TEE, ETC.);':'FITTING',
    },
    
    # dictionary for reconn
    'InspectionType':{
        'nbd':'NBD',
        'bd':'BD',
    },
    
    'BuildingAccessible':{
        'yes':'YES',
        'no':'NO',
    },
    
    'PressureRegulatorInBuilding':{
        'Yes':'YES',
        'no':'NO',
    },

    'TypeOfDwellingCd':{
        'SINGLE UNIT DWELLING':'SINGLE',
        'COMMERCIAL':'COMMERCIAL',
        'MULTIUNIT': 'MULTIUNIT',
        'MULTI-UNIT RESIDENCE/DWEL':'MULTIUNIT',
        'INDUSTRIAL':'INDUSTRIAL',
        'SINGLE':'SINGLE',
        'Multi Unit (>=2Fam)': 'MULTIUNIT',
        'Commercial':'COMMERCIAL',
        '0':'',
        'Single Unit/Family':'SINGLE',
        'SCHOOL':'OTHER',
        'Residential':'MULTIUNIT',
        'PARKING GARAGE': 'OTHER',
        'Industrial':'INDUSTRIAL',
        'Hotel':'COMMERCIAL',
        'Commercial, multi unit, hotel':'COMMERCIAL',
        'Church':'OTHER',
        'School':'OTHER',
        'Bar':'COMMERCIAL',
        'School, NYU':'OTHER',
        'Adult daycare': 'COMMERCIAL',
        'Treatment facility': 'INDUSTRIAL',
        'High school' :'OTHER',
        'PARK':'OTHER',
        'Chinese Consulate':'COMMERCIAL',
        'Baruch college' :'OTHER',
        'Empire State Building': 'COMMERCIAL',
        'Nycha':'COMMERCIAL',
        'Church':'OTHER',
        'Fota mosque': 'OTHER',
        'Harlem healthcare center':'COMMERCIAL',
        'Super market':'COMMERCIAL',
        'Building under construction' :'OTHER',
        'Church and school':'OTHER',
        'Hotel':'COMMERCIAL',
        'Hotel shelter' :'COMMERCIAL',
        'Hotel/shelter':'COMMERCIAL',
        'Residential and commercial' :'COMMERCIAL',
        'Community center':'COMMERCIAL',
        'Shelter  EconoLodge':'COMMERCIAL',
        'Shelter' :'COMMERCIAL',
        'Hotel, movie theater and stores':'COMMERCIAL',
        'Multiunit residential and commercial unit': 'MULTIUNIT',
        'Shelter- 99 rooms':'COMMERCIAL',
        'Commercial and residential':'COMMERCIAL',
        'Women’s shelter':'COMMERCIAL',
        'Hotel and Restaurant': 'COMMERCIAL',
        'Church and school' :'OTHER',
        'Commercial and apartments':'COMMERCIAL',
        'Clinical' :'COMMERCIAL',
        'Police station':'OTHER',
        'Kitchen':'COMMERCIAL',
        'Convent' :'OTHER',
        'Residential':'MULTIUNIT',
        'School building': 'OTHER',
        'Apartment' :'MULTIUNIT',
        'Lodge' :'MULTIUNIT',
        'Commercial and residential share line':'COMMERCIAL',
        'Other':'OTHER',
        'Commercial N MULTI UNIT SERVICE 3 APTS':'COMMERCIAL',
        'Public park' :'OTHER',
        'College' :'COMMERCIAL',
        'Rooming house' :'MULTIUNIT',
        'Multiple units and commercial' :'COMMERCIAL',
        'Multi units and commercial' :'COMMERCIAL',
        'Multiple units and commercial':'COMMERCIAL',
        'Park department' :'OTHER',
        'Daycare' :'COMMERCIAL',
        'Housing for the elderly' :'COMMERCIAL',
        'Restaurant' :'COMMERCIAL',
        'Post Office':'OTHER',
    },

    'MeterLocation':{
        'BASEMENT':'BASEMENT',
        'METER ROOM':'METERROOM',
        'meterroom':'METERROOM',
        # 'METER AT/NEAR POE':'',
        # 'METERPOE':'',
        # 'METERSREMOTE':'',
        # 'ROOMSET':'',
        'Meter at/near POE':'METERROOM',
        'Mtrs in Mult. Remote Locations':'OTHER',
        'Apartments ':'OTHER',
        'Meter Room':'METERROOM',
        '0':'',
        'Room Set':'OTHER',
        '5C kitchen ':'OTHER',
        '5G kitchen ':'OTHER',
        '4H kitchen ':'OTHER',
        'apartments':'OTHER',
        '4a kitchen ':'OTHER',
        '3c kitchen ':'OTHER',
        'Apt':'OTHER',
        '3F kitchen ':'OTHER',
        ' B35 kitchen ':'OTHER',
        '51B kitchen ':'OTHER',
        'Apartment ':'OTHER',
        'Basement kitchen ':'OTHER',
        'Kitchen basement ':'BASEMENT',
        'Garage ':'GARAGE',
        'Apt 3  kitchen ':'OTHER',
        'Apt 33 kitchen ':'OTHER',
        'Basement to your left before reaching POE':'BASEMENT',
        '4B kitchen ':'OTHER',
        'Apt 3A kitchen ':'OTHER',
        'No meters in the building.':'',
        '2D kitchen ':'OTHER',
        '3E kitchen ':'OTHER',
        '6D kitchen ':'OTHER',
        '6F kitchen ':'OTHER',
        'Meter located in a basement apartment. Found off and locked':'BASEMENT',
        '4L kitchen ':'OTHER',
        '3M kitchen ':'OTHER',
        'Kitchen apt 5C':'OTHER',
        '2B kitchen ':'OTHER',
        '1L kitchen ':'OTHER',
        'Kitchen ':'OTHER',
        '1K kitchen ':'OTHER',
        '1D kitchen ':'OTHER',
        'Apart 1RS':'OTHER',
        'In apartment 16':'OTHER',
        '4G kitchen ':'OTHER',
        'Meter in basement':'BASEMENT',
        'No meters':'',
        'Head of service outside':'OUTSIDE',
        'Apt 4F kitchen ':'OTHER',
        ' Apt 3F Kitchen wall ':'OTHER',
        'apt ':'OTHER',
        'METERS ON THIS TICKET WERE NOT LOCATED.':'',
        'Kitchen cabinet ':'OTHER',
        'Unknown ':'',
        'Walk way In apartment ':'HALLWAY',
        'outside meter':'OUTSIDE',
        'Apartment':'OTHER',
        'In apartments ':'OTHER',
        'Apartment 2':'OTHER',
        'Apartment 53':'OTHER',
        'Meter in garage. Poe in room behind meter ':'GARAGE',
        'Basement ':'BASEMENT',
        'Cabinet':'OTHER',
        'Kitchen':'OTHER',
        'Meters are located in each apartment ':'OTHER',
        'Meter not found ':'',
        'Bedroom':'OTHER',
        'THERE ARE NO METERS IN BUILDING ':'',
        'No meter or service at this location ':'',
        'Under stoop meter room ':'METERROOM',
        'Basement':'BASEMENT',
        'NO METERS, BUILDING IS VACANT.':'',
        'meters removed ':'',
        'laundry room':'OTHER',
        'meters are in Apts ':'OTHER',
        'Meters in basement POE INACCESSIBLE':'BASEMENT',
        'Boiler room upstairs':'OTHER',
        '2nd floor':'OTHER',
        'closet in living room infront of bathroom':'OTHER',
        'outside':'OUTSIDE',
        'Kitchen cabinet':'OTHER',
        'No meter found ':'',
        'garage':'GARAGE',
        'Unknown':'OTHER',
        'Doorway of chuch ':'OUTSIDE',
        'closet in room next door from POE':'OTHER',
        'No meter found':'',
        'meters in garage':'GARAGE',
        'THERE IS NO METERS IN THIS LOCATION.':'',
        'meter removed ':'',
        ' Basement ':'BASEMENT',
        'METER NO LONGER HERE. Inlet and outlet capped. Head of service capped and OFF. ':'',
        'Living space ':'OTHER',
        '12 meters are in the apartment.4 meters bsmt ':'OTHER',
        'passage of Basement ':'BASEMENT',
        'in the basement near Boiler room':'BASEMENT',
        'Store ':'OTHER',
        'Both meters found disconnected and on floor ':'OTHER',
        'Restaurant basement ':'BASEMENT',
        'Meter in hallway and POE is under stairs ':'OTHER',
        'Basement apartment':'BASEMENT',
        'In garage ':'GARAGE',
        'SODA CLUB BASEMENT ':'BASEMENT',
        'Basement of 432 East 10th Street ':'BASEMENT',
        'There’s no NBD meters in this location ':'',
        'Vault ':'OTHER',
        'meters located in room next to POE':'OTHER',
        'No meters ':'',
        'Inside store':'OTHER',
        'No meter':'',
        'Meters are in apartments ':'OTHER',
        'Restaurant ':'STORE',
        'METERS LOCATED IN THE BASEMENT OF 302 Bleecker Street ':'BASEMENT',
        'Lab on the 1st FL':'OTHER',
        'Meters in BSMT in cabinets':'OTHER',
        'No gas meter at location':'',
        'In back of house':'OTHER',
        'Meter in next room over ':'OTHER',
        'In middle of basement ':'BASEMENT',
        'Basement bathroom':'BASEMENT',
        'Other  no gas service in building, no meters ':'',
        'Meter in Store':'STORE',
        'Outside in the front ':'OUTSIDE',
        'meter removed':'',
        'Meter in closet in back room ':'OTHER',
        'Outside Meter Set':'OUTSIDE',
        'In cabinet store ':'STORE',
        'No meters in building ':'',
        'up above in a small closet door in the back':'OTHER',
        'outside ':'OUTSIDE',
        'Meter not present ':'',
        'Behind the door to the right ':'OTHER',
        'meter closet in back':'OTHER',
        'Meter disconnected on floor':'OTHER',
        'meters in separate room from POE':'OTHER',
        'loading dock':'OTHER',
        'In a separate meter room ':'METERROOM',
        'No meters in building. Building under construction ':'',
        'Meter was removed new service in process . New sleeve already installed ':'',
        'Removed by coned ':'',
        '8 ft above POE':'OTHER',
        '15 ft from POE':'OTHER',
        'Separate basement from POE ':'BASEMENT',
        '12 ft from POE':'OTHER',
        '12 ft from POE ':'OTHER',
        '10 ft from POE ':'OTHER',
        'Apartment kitchen ':'OTHER',
        'Left side of boiler room entrance ':'OTHER',
        'Apartment living room ':'OTHER',
        'Under stoop ':'OTHER',
        '15ft from POE ':'OTHER',
        'Apartment bathroom ':'OTHER',
        '40 ft from POE':'OTHER',
        '20 ft from POE in adjacent basement ':'BASEMENT',
        '10 ft from POE':'OTHER',
        '25 ft from POE ':'OTHER',
        'Basement hallway, close to stairs ':'BASEMENT',
        '25 feet from POE ':'OTHER',
        'Above apartment door inside':'OTHER',
        'Apartment closet':'OTHER',
        'Next to basement entrance ':'BASEMENT',
        'Ground floor next to supply closet, 30 ft from POE ':'OTHER',
        'Separate room, 8 ft from POE ':'OTHER',
        '15 ft from POE ':'OTHER',
        '20 ft from POE':'OTHER',
        '20 ft from POE ':'OTHER',
        '20 ft below POE':'OTHER',
        'Basement hall ':'BASEMENT',
        '13 ft from POE':'OTHER',
        '12 ft from POE and basement front ':'BASEMENT',
        'Storage room ':'OTHER',
        'Basement front ':'BASEMENT',
        '8 ft front of basement entrance ':'BASEMENT',
        'Right of basement entrance ':'BASEMENT',
        'Basement hall':'BASEMENT',
        ' Back of basement ':'BASEMENT',
        'Bathroom ':'OTHER',
        'Kitchen cabinet above refrigerator ':'OTHER',
        'Next to basement stairs ':'BASEMENT',
        'Water meter room ':'METERROOM',
        'Inside store above atm ':'STORE',
        'No Access ':'',
        'In apartments':'OTHER',
        'One meter in apartment 4 d':'OTHER',
        'Meter next to Poe and 4 meters are in the ceiling ':'OTHER',
        'Sub basement ':'BASEMENT',
        'Under the stairs ':'OTHER',
        'In apartment':'OTHER',
        'Meters are in the basement of the garage around the corner ':'BASEMENT',
        'In laundry room ':'OTHER',
        'Cabinet by entryway ':'OTHER',
        'Cabinets ':'OTHER',
        'HALLWAY OF BASEMENT':'HALLWAY',
        '3 FLR':'OTHER',
        'In room under POE FLOOR C1':'OTHER',
        'Apartment cabinet':'OTHER',
        'Poe in a no access crawl space ':'OTHER',
        'Hallway ':'HALLWAY',
        'All NBD Gas Meters Removed From Building':'',
        'No meter in building ':'',
        'No NBD Gas Meters in Building Converted to Electric':'',
        'No gas in building ':'',
        'Hallway near boiler room ':'HALLWAY',
        'Vault':'OTHER',
        '14 Floor':'OTHER',
        'Hallway':'HALLWAY',
        'Rainbow Room 65th Floor':'OTHER',
        'Restroom':'OTHER',
        'meter closet ':'METERROOM',
        'Meter is in hallway POE is in basement apartment ':'BASEMENT',
        'Apt kitchen':'OTHER',
        'No meter at location ':'',
        'No meter locked at hos customer waiting on meter to be installed ':'',
        'Behind counter inside car wash ':'',
        '1st fl hallway ':'HALLWAY',
        'Hallway closet first floor':'HALLWAY',
        'Basement bathroom ':'BASEMENT',
        'Apartment 16':'OTHER',
        'Kitchen closet':'OTHER',
        'Basement office':'BASEMENT',
        'Other side of basement ':'BASEMENT',
        'Closet near basement door':'BASEMENT',
        'Closet':'OTHER',
        'Meters in vaulted area outside set ':'OUTSIDE',
        'No meters on site. Off and capped and service valve.':'',
        'laundry room ':'OTHER',
        'Basement meter room':'BASEMENT',
        'Meter in apt ':'OTHER',
        'Apartment 10d':'OTHER',
        'CUSTOMER APARTMENT ':'OTHER',
        'Customers Apartment ':'OTHER',
        'Apartment kitchen':'OTHER',
        'Meters are in the Apartments ':'OTHER',
        'In basement ':'BASEMENT',
        'In apartment 5E':'OTHER',
        'Pump Room':'OTHER',
        'warehouse ':'OTHER',
        'Outside under stairs':'OUTSIDE',
        '11 Floor Hallway ':'HALLWAY',
        'Bsmt':'BASEMENT',
        'Separate from POE':'OTHER',
        'Hall way':'HALLWAY',
        'Meter in kitchen ':'OTHER',
        'All meters in apts except for the 5 listed below ':'OTHER',
        'All meters in apts except for boiler dryer and super meters':'OTHER',
        'All meters in apts except for laundry room ':'OTHER',
        'Kitchen of apt':'OTHER',
        'One meter in apt apt 2 second meter located in REST BASEMENT ':'BASEMENT',
        'Ground floor':'OTHER',
        'In ceiling removal panel':'OTHER',
        'crawl space ':'OTHER',
        '1st floor behind sliding doors':'OTHER',
        'Behind wall in store':'STORE',
        'Basement hallway ':'BASEMENT',
        'Apartment 7a':'OTHER',
        'In apartment ':'OTHER',
        'In apartment 12A':'OTHER',
        'Basement. ':'BASEMENT',
        'Behind basement wall':'BASEMENT',
        'Other':'OTHER',
        'Separate Basement':'BASEMENT',
        'Bsmt oil tank room':'BASEMENT',
        'Basement of 72nd gourmet deli':'STORE',
        'In apartment 2G kitchen ':'OTHER',
        'Locker room':'OTHER',
        'Back right of steam room':'OTHER',
        'Behind Sheetrock ':'OTHER',
        'Couldn’t locate ':'',
        'Apts':'OTHER',
        'Meter in apartment ':'OTHER',
        'Meter in garage on other side of wall from Poe ':'GARAGE',
        'Basement west side':'BASEMENT',
        'Poe in garage, meter in BSMT apartment ':'GARAGE',
        'ground floor ':'OTHER',
        'In each apartment ':'OTHER',
        'Apt 53':'OTHER',
        'Basement closet by the front door':'BASEMENT',
        'Ground floor wall panel ':'OTHER',
        'Closet in hallway ':'OTHER',
        'Locker room ':'OTHER',
        'Basement closets':'BASEMENT',
        'Basement boiler room':'BASEMENT',
        'Basement of 502 w 139th ':'BASEMENT',
        'Closet ':'OTHER',
        'Close to outside ':'OTHER',
        'meters are in the boiler  the POE  is in the inaccessible business next door ':'OTHER',
        'last door on the right of the front door inside of the bathroom ':'OTHER',
        'Bedroom Closet':'OTHER',
        'Apt 26 kitchen ':'OTHER',
        'Apt kitchen ':'OTHER',
        'Apt 18 kitchen ':'OTHER',
        '12A kitchen ':'OTHER',
        'Apt 1 kitchen ':'OTHER',
        '2W kitchen ':'OTHER',
        '1E kitchen ':'OTHER',
        'apt 5b kitchen':'OTHER',
        '3G kitchen ':'OTHER',
        '6L kitchen ':'OTHER',
        '4D kitchen ':'OTHER',
        '2N kitchen ':'OTHER',
        '6R kitchen ':'OTHER',
        '2E kitchen ':'OTHER',
        '5 M kitchen ':'OTHER',
        '5k kitchen ':'OTHER',
        'Kitchen  2L':'OTHER',
        '41 A kitchen ':'OTHER',
        '41A kitchen ':'OTHER',
        '33A kitchen ':'OTHER',
        '23B kitchen ':'OTHER',
        '23 B kitchen ':'OTHER',
        'Bsmt ':'BASEMENT',
        'Apt 41 kitchen ':'OTHER',
        'Apt 1C kitchen ':'OTHER',
        'apartment #4A':'OTHER',
        'Apt 63 kitchen ':'OTHER',
        'Apt 1i kitchen ':'OTHER',
        '5E kitchen ':'OTHER',
        'Apt 65 kitchen ':'OTHER',
        '5D kitchen ':'OTHER',
        '5A kitchen ':'OTHER',
        '1A kitchen ':'OTHER',
        'Apt 9 kitchen ':'OTHER',
        'Apt 31 kitchen ':'OTHER',
        '12SE closet ':'OTHER',
        'Apt 12SE closet ':'OTHER',
        'In the kitchen of the restaurant ':'OTHER',
        'Meter in closet ':'OTHER',

    },

    'CorrosionFoundOnCdCustomer':{
        'HOUSE PIPE':'HOUSE_PIPE',
        'CUSTOMER PIPE':'OTHER',
        'GAS CARRIER PIPE':'OTHER',
        'Gas Carrier Pipe':'HOUSE_PIPE',
        'None':'NONE',
        '0':'',
        'Gas Carrier Pipe,Meter':'HOUSE_PIPE',
        'Gas Carrier Pipe,Service Head Adapter':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,Tee’s,elbows,fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar, Fittings, elbows, tees':'OTHER',
        'Gas Carrier Pipe,Sleeve':'HOUSE_PIPE',
        'Meter outlet ':'EXTENSION',
        'meter piping ':'HOUSE_PIPE',
        'Sleeve':'SLEEVE',
        'Service Head Adapter':'OTHER',
        'Gas Carrier Pipe,Regulator,Sleeve':'OTHER',
        'Gas Carrier Pipe,Regulator':'OTHER',
        'Service Head Adapter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Regulator,Service Head Adapter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Service Head Adapter,Sleeve':'OTHER',
        'None,Sleeve':'SLEEVE',
        'Gas Carrier Pipe,Regulator,Service Head Adapter':'OTHER',
        'no access ':'',
        'POE,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, fittings, elbows,meter bar':'OTHER',
        'Regulator,Sleeve':'OTHER',
        'Before head of service valve ':'EXTENSION',
        'Piping connections ':'EXTENSION',
        'poe ':'OTHER',
        'Meter controller valves':'EXTENSION',
        'No access to head of service ':'',
        'No access to the head of service ':'',
        'POE':'OTHER',
        'meter bars, swivels, fittings ':'OTHER',
        'Gas Carrier Pipe,None,Sleeve':'OTHER',
        'Meter inlet/outlet':'OTHER',
        'Meter, HOS':'OTHER',
        'meter bar, swivels, fittings ':'OTHER',
        'Meters':'OTHER',
        'Meter valve':'OTHER',
        'Meter piping and connections ':'OTHER',
        'POE, Meter ,Sleeve':'OTHER',
        'no access to con Ed pipe':'',
        'Meter Valve and Bar':'OTHER',
        'Before head of service POE':'OTHER',
        'Meter bar':'OTHER',
        'Meters ':'OTHER',
        'Meter':'OTHER',
        'Unknown':'OTHER',
        'POE, Meters,Sleeve':'SLEEVE',
        'Valve':'OTHER',
        'Link seal,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator,Tee’s,elbows,fitting meter bar':'OTHER',
        'extension pipe ':'HOUSE_PIPE',
        'POE, Meter, link seal,Sleeve':'OTHER',
        'Gas Carrier Pipe,POE,Sleeve':'OTHER',
        'POE, meters ,Sleeve':'OTHER',
        'Elbows, tee’s meter bars and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Meter ,Service Head Adapter':'OTHER',
        'Elbows, IJ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Regulator,Service Head Adapter,Sleeve':'OTHER',
        'Sleeve,fittings ':'SLEEVE',
        'Elbow,tee’s,fittings,meter bar,Gas Carrier Pipe,Meter':'OTHER',
        'None,Other':'OTHER',
        'Gas Carrier Pipe,None':'HOUSE_PIPE',
        'Gas Carrier Pipe,poe':'HOUSE_PIPE',
        'Gas Carrier Pipe,meter bars, swivels, fittings ':'OTHER',
        'Gas Carrier Pipe,Service Head Adapter,Sleeve,meter bar, swivels, fittings ':'OTHER',
        'No access':'',
        'Meter piping, fittings, meter bar':'OTHER',
        'Gas Carrier Pipe,meter bar, swivels, fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings,elbows,tee’s ':'OTHER',
        'Gas Carrier Pipe,Sleeve,meter bars, swivels, fittings ':'OTHER',
        'Regulator,Service Head Adapter,Sleeve,swivels, fittings ':'OTHER',
        'Gas Carrier Pipe,Regulator,Sleeve,swivels ':'OTHER',
        'Sleeve,meter bars, swivels, fittings ':'OTHER',
        'Regulator':'OTHER',
        'POE, Meter':'OTHER',
        'Gas Carrier Pipe,Regular vent pipe,Sleeve':'OTHER',
        'Fitting, meter bars':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar,elbows,fitting ':'OTHER',
        'Meters , POE ,Sleeve':'OTHER',
        'Meter, POE,Sleeve':'OTHER',
        'boiler room':'OTHER',
        'Gas Carrier Pipe,Other':'OTHER',
        'Service piping ':'HOUSE_PIPE',
        'Fittings,Gas Carrier Pipe':'OTHER',
        'Other,Sleeve':'OTHER',
        'Meter Valve.':'EXTENSION',
        'Coned piping hidden behind sheet rock':'HOUSE_PIPE',
        'Meter inlet':'OTHER',
        'POE and METER BAR':'OTHER',
        'POE, Meters':'OTHER',
        'None,POE':'OTHER',
        'Meter bar,None':'OTHER',
        'Meter Bar’s':'OTHER',
        'Gas Carrier Pipe,Meter bar':'OTHER',
        'Regulator pipe ':'HOUSE_PIPE',
        'Meters ,Sleeve':'OTHER',
        'Meters,Sleeve':'OTHER',
        'Head of service ':'OTHER',
        'POE ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meters':'OTHER',
        'Gas Carrier Pipe,Poe ':'HOUSE_PIPE',
        'Gas Carrier Pipe,House pipe ':'HOUSE_PIPE',
        'Meter,Sleeve':'SLEEVE',
        'Sleeve,and POE':'SLEEVE',
        'Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Regulator,Service Head Adapter':'OTHER',
        'swivels ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings,meter bar':'OTHER',
        'Meter bars':'OTHER',
        'Meter ':'OTHER',
        'Other,Service Head Adapter':'OTHER',
        'Gas Carrier Pipe,head of service Valve ':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows, fitting ':'OTHER',
        'fittings ':'OTHER',
        'Meters, meter piping ':'OTHER',
        'Gas Carrier Pipe,Meter, fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter piping ':'OTHER',
        'Gas Carrier Pipe,Meter, meter bar’s, pipe supports and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar’s and fittings ':'OTHER',
        'Fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter Housing # 3032277':'OTHER',
        'Gas Carrier Pipe,Sleeve,meter bar, swivels, fittings ':'OTHER',
        'Meters and meter piping ':'OTHER',
        'Con ed piping hidden behind sheet rock. Not visible ':'OTHER',
        'Gas Carrier Pipe,meter bars, swivels, fittings. ':'OTHER',
        'Meter piping ,Service Head Adapter':'OTHER',
        'Fitting,meter bars,Gas Carrier Pipe,Sleeve':'OTHER',
        'House piping':'HOUSE_PIPE',
        'At the Poe gas carrier pipe ':'HOUSE_PIPE',
        'Meter piping ,Sleeve':'OTHER',
        'Meter outlet/inlet':'OTHER',
        'Mcv packing nut':'OTHER',
        'Mcv pack nut':'OTHER',
        'After hos ,Gas Carrier Pipe':'HOUSE_PIPE',
        'Unknown, I am unable to access Con Ed piping':'OTHER',
        'Gas Carrier Pipe,IJ':'OTHER',
        'Level 2 corrosion found before HSV,Sleeve':'OTHER',
        'Extension service,Gas Carrier Pipe':'OTHER',
        'Sleeve ': 'SLEEVE',
        'Piping outside': 'HOUSE_PIPE',
        'Vent pipe outside.': 'HOUSE_PIPE',
        'Gas Carrier Pipe,LEVEL 2 corrosion found on meter bar and near POE': 'OTHER',
        'Corrosion found at the Poe ,Gas Carrier Pipe': 'HOUSE_PIPE',
        'Meter elbow and piping':'OTHER',
        'did not inspect':'OTHER',
        'Gas Carrier Pipe,Meter ':'OTHER',
        'Fitting ,Gas Carrier Pipe,Service Head Adapter':'OTHER',
        'Fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Hidden behind sheet rock.  Not visible':'OTHER',
        'Fitting ,Gas Carrier Pipe':'OTHER',
        'Fitting ':'OTHER',
        'Gas Carrier Pipe,Meter bar,Sleeve':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fitted ':'OTHER',
        'Extension service, customer pipe and meter ':'OTHER',
        'Gas Carrier Pipe,Meter, meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meters, meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter bars':'OTHER',
        'Fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar’s and fittings ,Sleeve': 'OTHER',
        'Meter, fittings ':'OTHER',
        'Gas Carrier Pipe,Meters,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meters, meter bar’s and fittings ,Sleeve':'OTHER',
        'meter control valve ':'OTHER',
        'Meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,HOS,Sleeve':'OTHER',
        'Point of entry ':'OTHER',
        'Gas Carrier Pipe,Sleeve,Swivels, pipe supports ':'OTHER',
        'Gas Carrier Pipe,Swivel nut ':'OTHER',
        'Gas Carrier Pipe,Sleeve,poe ':'OTHER',
        'Gas Carrier Pipe,POE piping':'OTHER',
        'Elbows, tee’s, and fittings ,Gas Carrier Pipe':'OTHER',
        'Poe is off,plugged, and locked ':'OTHER',
        'Meter bar and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter, meter bar and fittings ':'OTHER',
        'Fitting,tee’s,elbows ,Gas Carrier Pipe,Meter':'OTHER',
        'None,did not inspect HOS and regulator':'OTHER',
        'Gas Carrier Pipe,Na':'HOUSE_PIPE',
        'Gas Carrier Pipe,Meter and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar and fittings ':'OTHER',
        'Sleeve,meter':'SLEEVE',
        'Sleeve,outlet swivel':'SLEEVE',
        'Gas Carrier Pipe,Swivel nut':'HOUSE_PIPE',
        'Meters, meter bar’s and fittings ,Sleeve':'OTHER',
        'Meters, meter bar’s and swivels,Sleeve':'OTHER',
        'Regulator,Sleeve,Vent pipe ':'OTHER',
        'Meter, meter bar and fittings ':'OTHER',
        'Meter piping and constructions':'OTHER',
        'Meter bar’s and fittings ,Sleeve':'OTHER',
        'Swivel nut ':'OTHER',
        'Corrosion ok. ConEd Piping/ HOS/ Regulator located on the outside. Extension pipe/ Gas meter located inside, Hallway, meeting room, behind cabinet':'OTHER',
        'Fittings, valves, meter bars,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter bars, fittings, valves ':'OTHER',
        'Fittings, valves, meter bars':'OTHER',
        'Valves, meter bars, fittings ':'OTHER',
        'Gas Carrier Pipe,Valves, meters, fittings ':'OTHER',
        'Gas Carrier Pipe,Valves ':'HOUSE_PIPE',
        'Gas Carrier Pipe,Service Head Adapter,Sleeve,Valves ':'OTHER',
        'Fittings, valve ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,HOS':'OTHER',
        'Fittings, valve,Gas Carrier Pipe':'OTHER',
        'Bars, fittings, valves ,Gas Carrier Pipe':'OTHER',
        'Fittings, meter bars, valves ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Valves, fittings, meter bars ':'OTHER',
        'Fittings, valves ,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, valves ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bars, fittings, valves ,Sleeve':'OTHER',
        'Fittings, valves, meter bars ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter bars, valves, fittings ':'OTHER',
        'Gas Carrier Pipe,Service valve':'OTHER',
        'Fittings, meter bar, valves ,Gas Carrier Pipe':'OTHER',
        'Fittings, meter bar, valves,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fittings, valve, coupling ,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, valve ,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bar, valve ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar, fittings, valves, coupling':'OTHER',
        'Bar, fittings, valves ,Gas Carrier Pipe,Meter':'OTHER',
        'Bar, fittings, valve,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Valves, fittings, bar':'OTHER',
        'Gas Carrier Pipe,Valve, meter bar, fittings':'OTHER',
        'Gas Carrier Pipe,Sleeve,Valve ':'OTHER',
        'Gas Carrier Pipe,Meter,meter bars, valves, fittings ':'OTHER',
        'Fittings, valves,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bars,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bar, valves ,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bar, valves,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bar,Gas Carrier Pipe,Meter':'OTHER',
        'Fittings, meter bar,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fittings,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fittings, meter bar, valve, elbows,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Service valve':'OTHER',
        'Meters, meter bar’s and fittings ':'OTHER',
        'Meter bar ':'OTHER',
        'Gas Carrier Pipe,Meter, meter bar and fittings ,Sleeve':'OTHER',
        'Meter connection ,Sleeve':'SLEEVE',
        'Gas Carrier Pipe,Meters, meter bar and fittings ':'OTHER',
        ' No access':'',
        'No access,None':'NONE',
        'Fittings , meters,Gas Carrier Pipe':'HOUSE_PIPE',
        'Gas Carrier Pipe,Pipe supports, meter bar and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar and fittings ,Sleeve':'OTHER',
        'Fitting, meter bar':'OTHER',
        'Gas Carrier Pipe,Pipe supports, meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter’s, meter bar’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter, pipe supports, meter bar’s and fittings ':'OTHER',
        'Sleeve,Swivels and fittings ':'OTHER',
        'Swivel nut':'OTHER',
        'Meter bar,tee’s,elbows,fittings ':'OTHER',
        'Gas Carrier Pipe,Pipe supports, meters, meter bar’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Poe,Service Head Adapter':'OTHER',
        'Sleeve and Coned pipe ':'OTHER',
        'HEAD OF SERVICE PIPPING 7/26/23':'OTHER',
        'Meter nut':'OTHER',
        'Fittings ,Service Head Adapter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Pipe supports, meter bar’s and fittings ,Sleeve':'OTHER',
        'Meter’s, meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meters, pipe supports, meter bar’s and fittings ':'OTHER',
        'Meters and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Head of service ':'OTHER',
        'Gas Carrier Pipe,Meter’s, meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Pipe supports, meter bar’s and fittings , meters':'OTHER',
        'Meters, meter bar’s and fittings ,None':'OTHER',
        'Gas Carrier Pipe,Meters, pipe supports, meter bar’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meters and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Pipe supports ,Sleeve':'OTHER',
        'meter bars, swivels, fittings':'OTHER',
        'Sleeve,meter bar, swivels ':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ':'OTHER',
        'Meter bar and fittings ,Sleeve':'OTHER',
        'Meter and pipe connections ':'OTHER',
        'Regulator vent pipe ':'OTHER',
        'Point of entry':'OTHER',
        'Gas Carrier Pipe,Meter’s and fittings ,Sleeve':'OTHER',
        'None,Service Head Adapter,Sleeve':'OTHER',
        'Meter’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter bar, meter piping ':'OTHER',
        'meter control valve, meter bar, swivels, fittings ':'OTHER',
        'meter bars, swivels  ,fittings ':'OTHER',
        'swivels, fittings ':'OTHER',
        'swivels,  fittings ':'OTHER',
        'Meter,Meter bar':'OTHER',
        'Fittings ,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar and fittings ':'OTHER',
        'Gas Carrier Pipe,Sleeve,fittings ':'OTHER',
        'Piping supports ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar’s and fittings ':'OTHER',
        'Meter,Meter piping ':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe supports, meter bar’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe supports, meter bar’s and fittings ':'OTHER',
        'Partial access ':'OTHER',
        'Gas Carrier Pipe,Head of service ,Sleeve':'OTHER',
        'meter bars ':'OTHER',
        'Sleeve,meter bar, swivels, fittings ':'OTHER',
        'Fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Meter bar, meter piping ':'OTHER',
        'Gas Carrier Pipe,House pipe,Sleeve':'OTHER',
        'Sleeve,poe':'SLEEVE',
        'Gas Carrier Pipe,Meter,Meter bar and fittings ,Sleeve':'OTHER',
        'meter, swivels, fittings ':'OTHER',
        'No gas in building ':'',
        'Gas Carrier Pipe,fittings':'OTHER',
        'Gas Carrier Pipe,Meter bar ,None':'OTHER',
        'Gas Carrier Pipe,Meter,meter bar, swivels and fittings ':'OTHER',
        'Gas Carrier Pipe,meter bar, swivels and fittings ':'OTHER',
        'meter bar, swivels and fittings ':'OTHER',
        'Extension service before the meter ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,fittings ':'OTHER',
        'inlet and outlet swivels ':'OTHER',
        'Gas Carrier Pipe,Meter,meter bar, swivels, fittings ':'OTHER',
        'Fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fitting ,Gas Carrier Pipe,Sleeve':'OTHER',
        'meter bars and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar’s and fittings ,Sleeve':'OTHER',
        'Fittings ,Gas Carrier Pipe,Regulator':'OTHER',
        'Meter,Meter bar and fittings ':'OTHER',
        'meter bars, swivels and fittings ':'OTHER',
        'meter bars, swivels and fittings':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fitting ':'OTHER',
        'Gas Carrier Pipe,Piping connections ':'OTHER',
        'meter bar, inlet and outlet swivels, fittings':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fittings':'OTHER',
        'Gas Carrier Pipe,Sleeve,poe':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fitting,meter bar':'OTHER',
        'Elbows, meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'Meter bar, swivel and fittings ':'OTHER',
        'Swivels and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows, fitting, meter bar':'OTHER',
        'Meter piping':'HOUSE_PIPE',
        'Meter,Meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator,Tee’s,Elbows, meter bar, fittings ':'OTHER',
        'Inlet and outlet swivels,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar, fittings ':'OTHER',
        'Meter bar, pipe support ,Sleeve':'OTHER',
        'Meter,Pipe supports, meter bar’s and fittings ,Sleeve':'OTHER',
        'Sleeve,fitting ':'OTHER',
        'Gas Carrier Pipe,Meter bar, fittings, inlet and outlet swivels ':'OTHER',
        'Gas Carrier Pipe,Meter piping meter bar':'OTHER',
        'Pipe support, meter bar and fittings ':'OTHER',
        'Elbows, tee’s, meter bar and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'POINT OF ENTRY ,Sleeve':'OTHER',
        'Swivel and pipe support ':'OTHER',
        'Meter,None':'NONE',
        'Coupling ,Meter':'OTHER',
        'Meter,Poe':'OTHER',
        'Elbows and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bars and fittings,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator,Sleeve':'OTHER',
        'Elbows, tee, pipe supports , meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Point of entry ':'OTHER',
        'Service ':'EXTENSION',
        'Elbows, tee’s, pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meters and fittings':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting ':'OTHER',
        'Elbows, pipe supports, meter bar and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Elbows, tee’s, fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Extension service, customer pipe, and Poe ':'OTHER',
        'Elbows ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar,fittings,elbows,tee’s':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting,meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings ':'OTHER',
        'On the elbow of piping ':'OTHER',
        'Gas Carrier Pipe,Meters and fittings ':'OTHER',
        'Meter,Tee’s,elbows,meter bar':'OTHER',
        'Elbows, meter bar’s and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Tee’s,fitting,elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting,meter bar,':'OTHER',
        'Swivel nuts':'OTHER',
        'Gas Carrier Pipe,None,poe ':'OTHER',
        'Tee’s,elbows ':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows fittings,meter bar':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fittings ':'OTHER',
        'Elbow customer piping':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar':'OTHER',
        'Elbows, pipe supports, tee’s, meter bar’s and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting,meter bars':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe extensions ':'OTHER',
        'Elbow connections and meter piping ':'OTHER',
        'Elbows connections and meter bar':'OTHER',
        'Other':'OTHER',
        'Elbows,fittings,tee’s ,Gas Carrier Pipe,Meter':'OTHER',
        'Extension service,customer pipe,meter and Poe':'OTHER',
        'Extension service,customer pipe,meter and Poe ':'OTHER',
        'Gas Carrier Pipe,Meter,Sleeve,Tee’s,Elbows,fittings,meter bar':'OTHER',
        'Gas Carrier Pipe,Vent pipe ':'HOUSE_PIPE',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting, meter bar':'OTHER',
        'Gas Carrier Pipe,Meter bar,fitting,tee’s,elbows ,Sleeve':'OTHER',
        'Customer piping and extension ,Gas Carrier Pipe':'OTHER',
        'Elbows, pipe supports,tee’s , meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Vent pipe ':'HOUSE_PIPE',
        'Elbows, meter bar and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Elbows, pipe supports, and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Head of service valve ':'OTHER',
        'Fitting,elbows,tee’s ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,Tee’s,elbows,fitting ':'OTHER',
        'Elbows, pipe supports, meter bar and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Elbows, pipe supports, meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Elbows, pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting meter bar':'OTHER',
        'Elbows, meter bar and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fitting, meter bar,tee’s,elbows ,Gas Carrier Pipe,Meter':'OTHER',
        'Elbows, IJ, pipe supports ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Level 2 corrosion on customer pipping gas fittings and POE ':'OTHER',
        'Level 2 corrosion on IJ 90 degree elbow fittings and service valve  no corrosion detected on con ed pipping ':'OTHER',
        'Elbows ,Gas Carrier Pipe':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Sleeve,Sleeve IJ and service valve ':'OTHER',
        'Fitting,meter bar,tee’s ,Gas Carrier Pipe,Meter':'OTHER',
        'Tee’s,elbows,fittings, meter bars':'OTHER',
        'Meter bar & inlet of meter':'OTHER',
        'Gas Carrier Pipe,Pipe support, meter bar, fittings, elbows ':'OTHER',
        'Elbows, tee’s,meter bar’s ,Gas Carrier Pipe':'OTHER',
        'Fitting on meter inlet':'OTHER',
        'Pipe support':'OTHER',
        'Meter fitting ':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,fitting,meter bars':'OTHER',
        'Gas Carrier Pipe,Meter,Other':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator,Tee’s,elbows,fitting ':'OTHER',
        'Gas Carrier Pipe,Regulator,Tee’s,elbows,fittings ':'OTHER',
        'Elbows, tee’s, and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Sleeve,poe ':'SLEEVE',
        'House piping ':'HOUSE_PIPE',
        'IJ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,tee’s,fitting,elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings, meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings,elbows ':'OTHER',
        'Elbow ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter bar,tee’s,elbows, fittings ':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ,Gas Carrier Pipe,Regulator,Sleeve':'OTHER',
        'Meter room ':'OTHER',
        'Elbows, pipe supports, meter bar and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter valve, meter bar and fittings..':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,fitting,elbows,meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator,Tee’s,elbows,fitting,meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,tee’s,fittings,elbows ':'OTHER',
        'Elbows, meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Sleeve,Tee’s, meter bar’s and fittings ':'OTHER',
        'House pipe ': 'HOUSE_PIPE',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fitting,elbows,tee’s ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,meter bar,fittings':'OTHER',
        'Meter swivel ':'OTHER',
        'Elbows and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'HOS valve ':'OTHER',
        'Elbows,tee’s,fitting ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe supports, elbows, meter bar’s and fittings ':'OTHER',
        'Fitting,tee’s,elbows,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Sleeve,Supports and fittings ':'OTHER',
        'Elbows ,Sleeve':'SLEEVE',
        'Elbows, tee’s, pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Meter,Valve, fittings ':'OTHER',
        'Fitting,tee’s,elbows.meter bar,Gas Carrier Pipe,Meter':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Poe ,Sleeve':'OTHER',
        'Elbow,Gas Carrier Pipe':'HOUSE_PIPE',
        'Elbows, meter bar’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Meter,Meter bar,fittings, tees':'OTHER',
        'Meter,Meter bar’s and fittings ,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meters bar’s and fittings ':'OTHER',
        'None,p.o.e ':'NONE',
        'Extension service meter and Poe ':'OTHER',
        'Gas Carrier Pipe,Hos valve':'OTHER',
        'Elbows, meter bar’s, fittings and pipe supports ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Pipe supports, and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter bar ,Sleeve':'OTHER',
        'Elbows, tee’s, meter bar’s and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Elbows, pipe supports and fittings ,Gas Carrier Pipe':'OTHER',
        'T fitting & meter bar':'OTHER',
        'Gas Carrier Pipe,HOS Valve, meter bar and fittings ,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings,':'OTHER',
        'Gas Carrier Pipe,Pipe supports, elbows, meter bar and fittings ,Sleeve':'OTHER',
        'Elbows, tee’s, and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Elbows, meter bar and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Meter nut , ':'OTHER',
        'Vent piping ':'HOUSE_PIPE',
        'Corrosion found on the spilt sleeve near the POE.,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bars,tee’s,fitting ,elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,tee’s,fittings,elbows':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe supports, elbows, meter bar and fittings ':'OTHER',
        'Elbows and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Swivel , meter bar’s and fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fitting,elbows, Tee’s ':'OTHER',
        'Elbows, tee’s, fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings,tee’s,elbows ':'OTHER',
        'Elbows. Pipe supports, meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Elbows, tee’s, pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Elbows, pipe supports, meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'HOSV':'OTHER',
        'Fitting':'OTHER',
        'HOSV Valve':'OTHER',
        'Elbows, IJ, tee’s, fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Fittings ,Meter,Sleeve':'OTHER',
        'Fitting,elbows,meter bar,tee’s,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, elbows,fitting ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings, tee’s,elbows ':'OTHER',
        'Elbows, pipe supports, ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Pipe supports and fittings,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee, elbows ':'OTHER',
        'Swivels, elbows, fittings ':'OTHER',
        'Meter bar’s and valves':'OTHER',
        'Meter swivel nut,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings,elbows,tees':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fitting,elbows,tees':'OTHER',
        'Fitting, elbows,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,elbows,fitting,Tees':'OTHER',
        'Elbows,meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Valve':'HOUSE_PIPE',
        'Elbows, IJ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Elbows, tee’s,pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bars and fittings ':'OTHER',
        'Elbows, fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,None,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Regulator':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bars, tee’s, fitting,elbows ':'OTHER',
        'Elbows, tee’s , pipe supports ,Gas Carrier Pipe':'OTHER',
        'All piping':'HOUSE_PIPE',
        'Elbows,tee’s,meter bar and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'after poe ':'OTHER',
        'Elbows, tee’s. Meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Piping':'HOUSE_PIPE',
        'Poe sleeve ,Sleeve':'SLEEVE',
        'Gas Carrier Pipe,Sleeve,elbows, tee’s, fittings ':'OTHER',
        'Vent pipe':'HOUSE_PIPE',
        'Meter bar ,None':'OTHER',
        'Elbows, fittings, ,Gas Carrier Pipe':'OTHER',
        'HOS':'OTHER',
        'Elbows, tee’s, meter bar and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Fittings,tee’s,elbows ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Tees,fitting, elbows ':'OTHER',
        'Gas Carrier Pipe,Tees,Elbows, Fittings':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows, pipe supports, meter bar,':'OTHER',
        'Fittings, elbows, pipe supports ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Sleeve, meter bar’s, fittings, elbows ':'OTHER',
        'Gas Carrier Pipe,Piping, elbows, meter bar’s, fittings ':'OTHER',
        'Sleeve,fittings':'OTHER',
        'Elbows, tee’s meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'Elbows,tee’s, fittings, meter bar,,Meter':'OTHER',
        'Gas Carrier Pipe,Pipe supports, elbows, tee’s, meter bar’s and fittings ':'OTHER',
        'Inlet and outlet of meter ,Meter':'OTHER',
        'Elbows and fittings ,Gas Carrier Pipe':'OTHER',
        'Customer piping ':'HOUSE_PIPE',
        'Elbows, tee’s, fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,Sleeve':'OTHER',
        'Elbows, fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, meter bar’s, fittings, elbows':'OTHER',
        'Other on gas piping ':'HOUSE_PIPE',
        'Elbows,tees, meter bar’s ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Meter bar, fittings ':'OTHER',
        'Elbows, tee’s and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Extension service, customer pipe, meter and Poe ':'OTHER',
        'Elbows, fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Tee’s,elbows,meter bar':'OTHER',
        'Elbows, Tees, pipe support ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter bar, Tee’s, Elbow, fittings':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, Fittings, elbows ':'OTHER',
        'Elbows, IJ, meter bar and fittings ,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows, fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbow,meter bar, Fitting’s ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, Elbows, Meter Bar,':'OTHER',
        'Elbows, tee’s,Gas Carrier Pipe,Sleeve':'OTHER',
        'Meter supports':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, Elbows, Fittings, Meter Bar':'OTHER',
        'Extension service,customer pipe,Poe and meter ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter Bar, Tee’s, elbows, Fittings ':'OTHER',
        'Gas Carrier Pipe,Pipe running from POE to meter inlet':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s , Elbows, fittings, Meter bar ':'OTHER',
        'Customer piping ,Gas Carrier Pipe,Regulator':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,Elbows,Meter bar,fittings ':'OTHER',
        'Elbows ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,Elbows, fittings, Meter Bar':'OTHER',
        'Poe sleeve ':'SLEEVE',
        'Gas Carrier Pipe,Meter,Tee’s, Elbows, fittings, meter bar ':'OTHER',
        'Elbows, IJ, meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'Elbows , meter bar and fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings, meter bar ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fittings,meter bars':'OTHER',
        'Elbows, IJ, meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, Meter bar, Fittings, Elbows ':'OTHER',
        'Fittings, Meter bar, elbows, ,Gas Carrier Pipe,Meter':'OTHER',
        'Elbow, tee’s, meter bar, fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar,elbows, fittings ':'OTHER',
        'Elbows, Pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Meter,Sleeve':'OTHER',
        'Elbows, IJ, meter bars and fittings ,Gas Carrier Pipe':'OTHER',
        'Elbows,tee’s, meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar, Fittings, Elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,meter bar, Fittings ':'OTHER',
        'Gas Carrier Pipe,Pipe supports, fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,None':'OTHER',
        'At the Poe ,Gas Carrier Pipe':'HOUSE_PIPE',
        'Elbows,Gas Carrier Pipe,Sleeve':'OTHER',
        'Elbows, meter bar and fittings ':'OTHER',
        'Gas Carrier Pipe,h. o. s ':'HOUSE_PIPE',
        'Gas Carrier Pipe,Meter,Tee’s,fittings,elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s, fitting, elbows, Meter bar':'OTHER',
        'Gas Carrier Pipe,Regulator vent pipe':'HOUSE_PIPE',
        'Gas Carrier Pipe,Regulator vent pipe ':'HOUSE_PIPE',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar,fittings,elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter Bar,elbows,tee’s, Fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,elbows,fitting':'OTHER',
        'Gas Carrier Pipe,Meter,Meter Bar, Tee’s, fittings, Elbows ':'OTHER',
        'Elbows, tees, meter bar’s and fittings ,Gas Carrier Pipe':'OTHER',
        'Fittings,Gas Carrier Pipe,Sleeve':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar,fitting, elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings,elbows, tee’s ':'OTHER',
        'Elbows, tees, meter bars and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Elbows, tee’s. Pipe supports, meter bar’s and fittings ,Gas Carrier Pipe,Meter':'OTHER',
        'Extension service,customer pipe, meters and Poe ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,fittings,elbows,meter bar':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,fittings, elbows, tee’s ,Regulator':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar,Fitting,elbows,tee’s':'OTHER',
        'Gas Carrier Pipe,Meter,Meter bar ,tee’s, elbows,fittings ':'OTHER',
        'Elbow ':'OTHER',
        'Gas Carrier Pipe,Meter,Pipe supports, fittings ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar,fittings, elbows ':'OTHER',
        'Gas Carrier Pipe,Meter,Tee’s,meter bar, fittings,elbows ':'OTHER',
        'Meter swivel ,Regulator':'OTHER',
        'Meter nut and swivel ':'OTHER',
        'Meter union':'OTHER',
        'Gas Carrier Pipe,Swivel':'OTHER',
        'Meter inlet and outlet ':'OTHER',
        'POE, Meter,Sleeve':'OTHER',
        'Gas Carrier Pipe,Point of entry ,Sleeve':'OTHER',
        'No access to Poe ':'',
        'meter inlet and outlet':'OTHER',
        'Con ed piping is not accessible room locked super unavailable.':'',

    },

    'POESleeveLocationCd':{
        'UNDER STAIRS':'UNDER_STAIRS',
        'Bsmt':'BASEMENT',
        'None':'NONE',
        'Outside ':'OTHER',
        '0':'',
        'Oil Tank Room':'OTHER',
        'Apt':'APARTMENT',
        'Garage':'GARAGE',
        'Hallway':'HALLWAY',
        'Understairs':'UNDER_STAIRS',
        'Outside':'OTHER',
        'Lower level basement. ':'BASEMENT',
        'ground floor ':'OTHER',
        'POE outside setting ':'OTHER',
        'Outside set':'OTHER',
        'Outside going into basement ':'BASEMENT',
        'Meter Room':'APARTMENT',
        'Vault':'VAULT',
        'Pottery Store Basement.':'BASEMENT',
        'loading dock':'OTHER',
        'GARAGE ':'GARAGE',
        'Utility room ':'OTHER',
        'Outiside bottom walk way.':'HALLWAY',
        'Bottom outside alley':'OTHER',
        'Laundry Mat Basement ':'BASEMENT',
        'Outside Poe ':'OTHER',
        'Not found ':'NONE',
        'METER VAULT':'VAULT',
        'No access to the POE':'',
        'Outside in the back yard':'OTHER',
        'POE is in basement near business meters ':'BASEMENT',
        'Poe comes from outside ':'OTHER',
        'Stoop':'UNDER_STAIRS',
        'Store ':'OTHER',
        'Corner store ':'OTHER',
        'Outside back of the house ':'OTHER',
        'PARKING GARAGE ':'GARAGE',
        'POE LOCATED OUTSIDE 10/27/23':'OTHER',
        'Basement ceiling ':'OTHER',
        'No access':'',
        'POE IN WALL NO ACCESS 10/25/23':'',
        'First floor':'OTHER',
        'Pottery Store Basement ':'BASEMENT',
        'obstructed ':'',
        'Upstairs boiler room':'OTHER',
        'Boiler Room':'OTHER',
        'ground floor':'OTHER',
        'Vault in HISPANIC MUSIC STORE.':'VAULT',
        '1st floor ':'OTHER',
        'utility room under stairs':'UNDER_STAIRS',
        'no access ':'',
        'Basement closet ':'BASEMENT',
        'Accessible under “ COPYLAND CENTER STORE” basement.':'BASEMENT',
        'No access store owner couldn’t provided access 10/18/23':'',
        'closet near entrance':'OTHER',
        'Unknown':'NONE',
        'Cellar':'BASEMENT',
        'Electrical room':'OTHER',
        'Room set accessed through the bathroom.  ':'OTHER',
        'Doorway of chuch':'OTHER',
        'Ground floor office ':'OTHER',
        'boiler room ':'OTHER',
        'Electric Room.':'OTHER',
        'Near locker room. Basement. (NOT SPRINKLER SYSTEM ROOM)':'OTHER',
        'Meter Room.':'OTHER',
        'front office ':'OTHER',
        'Two Poe one in basement, one in parking garage ':'OTHER',
        'meter room ':'OTHER',
        'Ground floor meter room ':'OTHER',
        'Gas meter room on the 1st floor ':'OTHER',
        'Trap door meter room ':'OTHER',
        'Ground floor wall panel':'OTHER',
        'Ground floor supermarket ':'OTHER',
        'store':'OTHER',
        'Outside on side of building next to stairs ':'UNDER_STAIRS',
        'In gym in closet ':'OTHER',
        'No acess':'',
        'back of warehouse':'OTHER',
        '1st floor utility room':'OTHER',
        'located on the first floor':'OTHER',
        'Behind access panel in office space in church basement ':'OTHER',
        'Street level, POE is a riser ':'OTHER',
        'Multiple POEs':'OTHER',
        'Gas meter room ground level ':'OTHER',
        'Gas meter room':'OTHER',
        'POE is in a crawl space':'OTHER',
        'no access to poe due to it being in a crawl space ':'OTHER',
        'Smoke Shop':'OTHER',
        'Front room street level ':'OTHER',
        'In store':'OTHER',
        'First floor meter room':'OTHER',
        '1st floor':'OTHER',
        'Behind Sheetrock not accessible ':'',
        'Not accessible, covered in Sheetrock':'',
        'First floor of dealership ':'OTHER',
        'HSV and regulator outside':'OTHER',
        'Not accessible ':'',
        'Hidden ':'NONE',
        'Corner of the building':'OTHER',
        'In cabinet in front room ':'OTHER',
        '1A Apts. Ground floor ':'OTHER',
        'Outside Meter Set':'OTHER',
        'closet in laundry room ':'OTHER',
        'Not found':'NONE',
        'Bsmt (Kitchen)':'BASEMENT',
        'Rec room of playground':'OTHER',
        'Gas Meter Room side of building':'OTHER',
        'Cafeteria Floor Panel':'OTHER',
        'In ceiling of store':'OTHER',
        'out front of the store':'OTHER',
        'meter closet':'OTHER',
        'Ground floor front storage room ':'OTHER',
        'Gas meter room 104':'OTHER',
        'Ground floor storage room ':'OTHER',
        'Can’t locate ':'NONE',
        'Outside going into building':'OTHER',
        'in backroom under shelve':'OTHER',
        'Room off drive way ':'GARAGE',
        'gas meter closet':'OTHER',
        'Bsmt concourse ':'BASEMENT',
        'Outside building':'OTHER',
        'POE located in neighboring building':'APARTMENT',
        'Kitchen ':'OTHER',
        'Basement of the building around the corner ':'BASEMENT',
        'Near back door ':'OTHER',
        'Fine fare supermarket ground floor ':'OTHER',
        'Women bathroom ':'OTHER',
        '2 poes 1 behind MLB Store 1 in Sprinkler room':'OTHER',
        'ConEd piping/HOS located outside ':'OTHER',
        'ConEd Piping/ HOS located outside, Meter/Extension Pipe located on the inside, basement.':'OTHER',
        'Located on the outside':'OTHER',
        'Meter Room/ Under Stoop ':'OTHER',
        'Basement apartment ':'BASEMENT',
        'Outside building ':'OTHER',
        'Under stoop ':'UNDER_STAIRS',
        'Under stairs outside ':'UNDER_STAIRS',
        'Store.':'OTHER',
        'Store behind cola machine ':'OTHER',
        'Office':'OTHER',
        'Unknown ':'',
        'Outside under stairs ':'OTHER',
        'Apartment ':'APARTMENT',
        'No access 8/1/23':'OTHER',
        '1275 1 Ave Bsmt':'APARTMENT',
        'Poe not accessible ':'',
        'On ground level in restaurant Poe is o riser':'OTHER',
        'In restaurant on ground level ':'OTHER',
        'first floor ':'OTHER',
        'In office next to window in a trap door ':'OTHER',
        'ground floor. door right. ':'OTHER',
        'No accesss':'',
        'Is in the basement of 590 207th street ':'APARTMENT',
        'warehouse mechanic ':'OTHER',
        'closet in kitchen ':'OTHER',
        'bsmt and garage':'OTHER',
        'closet in kitchen':'OTHER',
        'Inside restaurants ':'OTHER',
        'Street level riser ':'OTHER',
        'Boiler Room of Attached Building 550 W 50 St':'OTHER',
        'No accessibility  behind wall':'',
        'Outsiders ':'OTHER',
        'Sprinkler room ':'OTHER',
        'Lobby closet ':'OTHER',
        'Poe inaccessible '
        'Closet ':'OTHER',
        'Separate room ':'OTHER',
        'POINT OF ENTRY INACCESSIBLE DUE TO ITEMS STORED IN THE AREA':'',
        'No access 6/28/23':'',
        'None accessible 6/27/23':'',
        'Inaccessible ':'',
        'In a crawl space ':'OTHER',
        'Unable to access ':'',
        'No gas in building ':'',
        '500 E84 Bsmt':'BASEMENT',
        'Closet':'OTHER',
        'Unable to access 6/15/23':'',
        'Unable to access':'',
        'Utility closet':'OTHER',
        'Store behind door':'OTHER',
        'Unable to access no superintendent present 6/10/23':'',
        'Business main floor ':'OTHER',
        'Laundry Mart':'OTHER',
        'POINT OF ENTRY INACCESSIBLE ':'',
        ' Boiler room ':'OTHER',
        'In the office of the church ':'OTHER',
        'POE OUTSIDE UNDER STOOP 6/1/23':'OTHER',
        'Laundromat Store':'OTHER',
        'Laundromat ':'OTHER',
        'Laundromat store ':'OTHER',
        '1st Floor Laundromat ':'OTHER',
        'Fire pump room in the backyard.':'OTHER',
        'Store Bsmt':'BASEMENT',
        'In the basement of the bakery ':'BASEMENT',
        'Outside infront of the building.':'OTHER',
        'Outside near stairs':'UNDER_STAIRS',
        'auto shop':'OTHER',
        'auto shop ':'OTHER',
        'auto garage ':'GARAGE',
        'Closet In first floor':'OTHER',
        'Store front ':'OTHER',
        'Other':'OTHER',
        'In main building ':'OTHER',
        'Behind trap door inside wall ':'OTHER',
        'West side hatch ':'OTHER',
        'Bsmt Store':'BASEMENT',
        ' Meter room':'OTHER',
        'Meter room ground floor':'OTHER',
        'Closed to the outside ':'OTHER',
        'Unable to access POE ':'',
        'warehouse garage ':'GARAGE',
        'below trapped door':'OTHER',
        'warehouse ':'OTHER',
        'Ground FL':'OTHER',
        'Basement ':'BASEMENT',
        'Underneath store on the corner':'OTHER',
        'shop/room':'OTHER',
        'No access to POE and main gas valve':'',
        'Restaurant ':'OTHER',
        'Room closed to outside of building ':'OTHER',
        'Store front':'OTHER',
        'Electric meter room':'OTHER',
        'No access to poe':'',
        'Not Accessible':'',
        'REST BASEMENT ':'BASEMENT',
        'Side room ':'OTHER',
        'In water pump room ':'OTHER',
        'Next in store under stairs ':'OTHER',
        'Rear store':'OTHER',
        'Carpeting room':'OTHER',
        'Side room':'OTHER',
        'crawl space ':'OTHER',
        '2nd level kitchen ':'OTHER',
        'Meter located in the room on the left at 517 W 45 st the shoe store':'OTHER',
        'Unsure':'NONE',
        'Gas building outside of plaza':'OTHER',
        'Poe sealed ':'NONE',
        'Can not locate Poe ':'NONE',
        'Meter number ':'',
        'NA':'NONE',
        'Laundry room ':'OTHER',
        'Gas Room':'OTHER',
        'office ':'OTHER',
        'Store basement utility room':'BASEMENT',
        'Unable to located poe':'NONE',
        'Chu Ros Thai Basement':'BASEMENT',
        'Can not locate it ':'NONE',
        'Can not locate POE':'NONE',
        'Meter room inside office ':'OTHER',
        'In the parks restroom house.':'OTHER',
        'First floor near boiler':'OTHER',
        'Rec room of park':'OTHER',
        'Store office ':'OTHER',
        'Floor of meter room ':'OTHER',
        'Meter room only accessible from outside the building':'OTHER',
        'Behind office':'OTHER',
        'Next to main door ':'OTHER',
        'Near meter by entrance ':'OTHER',
        'Access panel  near entrance ':'OTHER',
        'Laundry room first floor':'OTHER',
        'Behind brick wall panel':'OTHER',
        'POE in sliding doors of entrance to restaurant ':'OTHER',
        'By the front door ':'OTHER',
        'Basement of Deli ':'BASEMENT',
        '1 Fl':'OTHER',
        'Ground level ':'OTHER',
        'Ground floor Apt kitchen ':'OTHER',
        'Laundry room ground floor ':'OTHER',
        'in back room ':'OTHER',
        'Meter closet as soon as U come thru the entrance ':'OTHER',
        'Gas room located at the side of the building ':'OTHER',
        'Taco Bell Basement':'BASEMENT',
        '1st near the security guard ':'OTHER',
        'Store Basement':'BASEMENT',
        'First floor closet ':'OTHER',
        'C Floor ':'OTHER',
        'Basement boiler room ':'BASEMENT',
        'Location separate from site of meter':'OTHER',
        'In connecting building ':'OTHER',
        '129A E97 st. Basement':'BASEMENT',
        'Under trap door ':'BASEMENT',
        'Basement laundry room':'BASEMENT',
        'Basement understood':'BASEMENT',
        'business in adjacent room if first floor property ':'OTHER',
        'Bsmt meter room':'BASEMENT',
        'Bsmt and garage ':'BASEMENT',
        'POE not found':'NONE',
        'In the Chinese restaurant ':'OTHER',
        'Poe outside going into basement':'BASEMENT',
        'Fence cage outside ':'OTHER',
        'Under stairs Outdoor':'UNDER_STAIRS',
        'No accessible, can’t determine.':'',
        'Not accessible, can’t determine.':'',
        'Basement separate room from meters ':'BASEMENT',
        'Blue mercury lobby, panel near door':'',
        'Blue Mercury lobby. Panel near door':'',
        'Office lobby ':'OTHER',
        'Front of store':'OTHER',
        'Basement closet':'BASEMENT',
        'Side of building in front ':'OTHER',
        'Doctors Office, box that is accessible and able to open covers the POE ':'OTHER',
        'Ground Floor Meter Room':'OTHER',
        'NOT ACCESSIBLE DURING THIS INSPECTION.':'',
        'POE located in office ceiling on first floor ':'OTHER',
        'Side of building outside ':'OTHER',
        'Meter room next to daycare':'OTHER',
        'Outside at 93rd Street entrance to the building ':'OTHER',
        'No access to POE super not on site ':'',
    },
    
    'PipingConstructionMaterialCd':{
        # 'METALLIC BARE':'METALLICBARE',
        # 'METALLIC COATED/PAINTED':'METALLICCOAT',
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
        '0':'',
        'Copper':'METALLICBARE',
        'Steel-Bare':'METALLICBARE',
        'Steel-Coated/Painted':'METALLICCOAT',
    },
    'MeterConstructionMaterialCd':{
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
        'METALLIC BARE':'METALLICBARE',
        'METALLIC COATED/PAINTED':'METALLICCOAT',
        'Steel-Coated/Painted':'METALLICCOAT',
        'Cast Iron-Coated/Painted':'METALLICCOAT',
        '0':'',
        'Steel-Bare':'METALLICBARE',
        'Cast Iron-Bare':'METALLICBARE',
    },
    'FittingsConstructionMaterialCd':{
        # 'METALLIC BARE':'METALLICBARE',
        # 'METALLIC COATED/PAINTED':'METALLICCOAT',
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
        'Steel-Bare':'METALLICBARE',
        'Steel-Coated/Painted':'METALLICCOAT',
        '0':'',
        'Aluminum-Coated/Painted':'METALLICCOAT',
        'Aluminum-Bare':'METALLICBARE',
    },
    'MainAndBranchService':{
        'YES':'YES',
        'NOT AVAILABLE':'NO',
        'NO':'NO',
        'yes':'YES',
        'no':'NO',
        '0':'',
    },
    'ServiceUnderStoop':{
        'yes':'YES',
        'no':'NO',
        '0':'',
    },
    'AccessibleCustomer':{
        'yes':'YES',
        'no':'NO',
        '0':'',
    },
    'NoAccessReasonCdCustomer':{
        'CATS':'CATS',
        'CUSTFAILAPPT':'CUSTFAILAPPT',
        'CUSTNOTHOME':'CUSTNOTHOME',
        'DID NOT ATTEMPT':'DIDNOTATTEMPT',
        'CORONAVIRUS':'CORONAVIRUS',
        'ADDRESSNOTEXIST':'ADDRESSNOTEXIST',
        'QUARANTINED':'QUARANTINED',
        'Cusotmer Refused Access':'CUSTREFUSE',
        'Did Not Attempt':'OTHER',
        'CATS Remarks':'CATS',
        'Customer Not at Home':'CUSTNOTHOME',
        'Customer Failed to Keep Appt.':'CUSTFAILAPPT',
        'Access Denied COVID-19':'CORONAVIRUS',
    },
    'CorrosionSeverityCdCustomer':{
        '1-NO CORROSION SEVERITY':'NONE',
        '2-LOW CORROSION SEVERITY':'LOW',
        '3-MEDIUM CORROSION SEVERITY':'MEDIUM',
        'Low Corrosion':'LOW',
        'None or Very Minimal':'NONE',
        '0':'',
        'High Corrosion':'HIGH',
        'Medium Corrosion':'MEDIUM',
    },
    'GasTurnedOff':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'AllServicePipingInspected':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'ImproperConditionsFoundConEd':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'AccessibleConEd':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'InspDoneOnAllSvcsEnteringBldg':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'ImproperSubstandardConditionConEdCds':{
        'Improper inside meter location': 'INSIDE_MTR_LOC_IMPR',
        'Inaccessible meter sets':'OTHER',
        'Inappropriate or substandard parts or materials':'PART_MATERIAL',
        ' Jurisdictional pipe not installed to utility specifications':'JURISDICTION_PIPING',
        'Meter set(s) without sufficient support':'OTHER',
        'Meter sets in a hot or humid environment':'OTHER',
        'Meter sets in an environment where they are in danger of damage from falling items around them':'DAMAGE_GAS_METER',
        'Meter sets without access to shut-off valves':'NO_SHUTOFF_VALVE',
        ' No regulator vent terminus':'REGULATOR_VENT_GAS',
        'Structures over a gas facility':'GAS_FCLTY_DEPTH_INCO',
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'UnauthorizedMetersFound':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'PlasticExposed':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'PlasticExtendsIntoStructure24Inches':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'SupervisorNotified':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'CorrosionSeverityCdConEd':{
        '1-NO CORROSION SEVERITY':'NONE',
        '2-LOW CORROSION SEVERITY':'LOW',
        '3-MEDIUM CORROSION SEVERITY':'MEDIUM',
        'Low Corrosion':'LOW',
        'None or Very Minimal':'NONE',
        '0':'',
        'High Corrosion':'HIGH',
        'Medium Corrosion':'MEDIUM',
    },
    'PoeAccessible':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'PoeNoAccessReasonCd':{
        'Job did not require access':'ACCESS_NOT_REQUIRED',
        'No Access':'NO_ACCESS',
        'CM Refusal':'CUST_REFUSED_ACCESS',
        'ACCESS NOT REQUIRED':'ACCESS_NOT_REQUIRED',
        'AOC CONDITION':'OTHER',
        'CUST REFUSED ACCESS':'CUST_REFUSED_ACCESS',
        'NO ACCESS':'NO_ACCESS',
    },
    'PoeFoundSealedCondition':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'PoeLeftSealedCondition':{
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    
    'PoeSealTypeCd':{
        'Duct Seal':'DUCT_SEAL',
        'Foam':'FOAM',
        'POE WAS SEALED AND PAINTED BY GDS, THEY MARKED THE SLEEVE AS LEVEL 2':'OTHER',
        'GDS mechanic will comeback to the property to seal the POE.':'OTHER',
        'Silicone':'SILICONE',
    },
    'GasOdorPresent':{
        'Yes (Call GERC)':'YES',
        '0':'',
        'yes':'YES',
        'no':'NO',
    },
    'GasLeakCorrosionRelated':{
        'no':'NO',
        'yes':'YES',
    },

}

In [ ]:

new_df['WR#']=first_df['ticket_number']
new_df['Borough'] = first_df['borough']
new_df['Muni'] = first_df['address_sub_admin_area']
new_df['InspectionType']=first_df['inspection_type']
new_df['CompletedBy'] = first_df['_assigned_to']
new_df['StartedDateTime']=first_df['_server_cr']
new_df['AssignedDateTime']=first_df['_server_updated_at']
new_df['NoOfMeters']=first_df['number_of_meters']
new_df['NoOfApartments']=first_df['number_of_apts']
new_df['AssetRemarks']=first_df['remarks']
new_df['GasLeakReadingCGI']=first_df['cgi_reading_gas']
new_df['GasLeakTicketNbr']=first_df['leak_ticket_']
new_df['LeakItemRemarks']=first_df['remarks5']

TypeOfDwellingCd_map = value_mapping.get('TypeOfDwellingCd',{})
new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].apply(lambda x: '' if x == 0 else 'SINGLE' if 'single' in str(x).lower() else TypeOfDwellingCd_map.get(x,'OTHER') if x and pd.notna(x) else 'SINGLE')

MeterLocation_map = value_mapping.get('MeterLocation',{})
new_df['MeterLocation'] = new_df['MeterLocation'].apply(lambda x:'' if x == 0 else 'BASEMENT' if 'basement' in x.lower() else 'HALLWAY' if 'hallway' in x.lower() else 'GARAGE' if 'garage' in x.lower() else MeterLocation_map.get(x,'OTHER') if x and pd.notna(x) else 'BASEMENT')

PipingConstructionMaterialCd_map = value_mapping.get('PipingConstructionMaterialCd',{})
new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].apply(lambda x: PipingConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

MeterConstructionMaterialCd_map = value_mapping.get('MeterConstructionMaterialCd',{})
new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].apply(lambda x: MeterConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

FittingsConstructionMaterialCd_map = value_mapping.get('FittingsConstructionMaterialCd',{})
new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].apply(lambda x: FittingsConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x,'OTHER') if x and pd.notna(x) else 'YES')

MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x,'OTHER') if x and pd.notna(x) else 'YES')

for column, mapping in value_mapping.items():
    if column != 'WR#' or column != 'InspectionType' or column !='StatedDateTime' or column != 'AssignedDateTime' or column != 'NoOfMeters' or column != 'NoOfApartments' or column != 'AssetRemarks' or column != 'GasLeakReadingCGI' or column != 'GasLeakTicketNbr' or column != 'LeakItemRemarks':
        new_df[column] = new_df[column].replace(mapping)

new_df = new_df[first_df['outside_set']!='YES']
new_df = new_df[first_df['accessible']!='NO']
            
new_df.to_excel('reconn_parsed.xlsx',index=False)

In [ ]:
print(new_df['TypeOfDwellingCd'])